# `Практикум по программированию на языке Python`

## `Задание 03. Рекуррентные Нейронные Сети. Dropout. LM`

#### Фамилия, имя:

Дата выдачи: <span style="color:red">__18 марта 23:59__</span>.

Мягкий дедлайн: <span style="color:red">__31 марта 05:00__</span>.

Стоимость: __10 баллов__ (основная часть заданий) + __7 баллов__ (дополнительные задания).

<span style="color:red">__В ноутбуке все клетки должны выполняться без ошибок при последовательном их выполнении.__</span>

#### `Москва, 2025`

Данное задание будет состоять из двух частей:
1. Применение рекуррентной сети для решения задачи классификации текста. Более конкретно -- предсказания рейтинга отзыва фильма.
2. Простейшая лингвистическая модель для генерации текста на основе LSTM.

При выполнении задания вы обучите LSTM с разным уровнем "коробочности", а также познакомитесь с различными способами применения DropOut к рекуррентным архитектурам. В рекуррентных архитектурах вариантов, куда можно наложить бинарную маску шума, гораздо больше, чем в нейросетях прямого прохода.

Во второй части вы попробуете реализовать простейший рекуррентный декодер для генерации текстов.

Задание сделано так, чтобы его можно было выполнять на CPU, однако RNN - это ресурсоёмкая вещь, поэтому на GPU с ними работать приятнее. Можете попробовать использовать [https://colab.research.google.com](https://colab.research.google.com) - бесплатное облако с GPU.

**Для корректного отображения картинок, вам может понадобится сделать ноутбук доверенным (Trusted) в правом верхнем углу**

# `Часть 0. Загрузка и предобработка данных (1 балл)`

## `Рекомендуемые гиперпараметры`

In [5]:
max_length = 200
top_n_words = 5000

hidden_dim = 128
embedding_dim = 32

num_epochs = 15
batch_size = 64
learning_rate = 1e-3

Первое, что нужно сделать — скачать, предобработать данные и организовать их таким образом, чтобы их можно было подавать в нейронную сеть.

Для обеих частей задания мы будем использовать [**Large Movie Review Dataset**](https://ai.stanford.edu/~amaas/data/sentiment/).

## `Загрузка и предобработка данных`

In [6]:
import os

Загрузите данные по ссылке выше. (**tip**: используйте `wget`)

In [7]:
if not os.path.exists("./aclImdb_v1.tar.gz"):
    os.system("wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz")
else:
    print("File already exists.")


Распакуйте скачанные данные в папку `aclImdb` (**tip:** используйте `tar`)

In [8]:
if not os.path.exists('aclImdb'):
    os.system("tar -xzf aclImdb_v1.tar.gz")
else:
    print("File already extracted.")

Посмотрите в файле `./aclImdb/README` как организованы данные:

In [9]:
! cat ./aclImdb/train/pos/10003_8.txt

This is not the typical Mel Brooks film. It was much less slapstick than most of his movies and actually had a plot that was followable. Leslie Ann Warren made the movie, she is such a fantastic, under-rated actress. There were some moments that could have been fleshed out a bit more, and some scenes that could probably have been cut to make the room to do so, but all in all, this is worth the price to rent and see it. The acting was good overall, Brooks himself did a good job without his characteristic speaking to directly to the audience. Again, Warren was the best actor in the movie, but "Fume" and "Sailor" both played their parts well.

In [10]:
test_data_path = './aclImdb/test/'
train_data_path = './aclImdb/train/'

In [11]:
import random
from tqdm import tqdm
from functools import partial
from collections import defaultdict
from typing import Optional, Tuple, Union, List

from IPython.display import Markdown, display


import nltk
nltk.download('stopwords')

import regex
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader

import tokenizers
from tokenizers import Tokenizer, trainers, pre_tokenizers

torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(False)

torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)
torch.autograd.set_detect_anomaly(False)

torch.set_float32_matmul_precision('high')
torch.backends.cuda.matmul.allow_tf32 = True

os.environ["TOKENIZERS_PARALLELISM"] = "false"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
def set_global_seed(seed: int) -> None:
    """Set global seed for reproducibility.
    :param int seed: Seed to be set
    """
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def check_numel(module: torch.nn.Module, params_numel: int, buffers_numel: Optional[int] = None) -> None:
    """Check whether module has correct number of parameters and buffers
    :param torch.nn.Module module: Target model
    :param int params_numel: Target number of parameters
    :param Optional[int] buffers_numel: Target number of buffers
    :rtype:
    """
    numel = sum(param.numel() for param in module.parameters())
    assert numel == params_numel, f'For params numel != correct numel: {numel} vs {params_numel}'

    if buffers_numel is not None:
        numel = sum(param.numel() for param in module.buffers())
        assert numel == buffers_numel, f'For buffers numel != correct numel: {numel} vs {buffers_numel}'


In [13]:
set_global_seed(42)

Стандартной предобработкой данных является токенизация текстов. Полученные токены можно будет закодировать и затем подавать на вход нейронной сети. Ключевым моментом, который влияет на скорость работы нейросети и её размер в памяти — размер словаря, используемого при токенизации. Для задачи классификации мы можем убрать часть слов (стоп слова, редкие слова), ускорив обучение без потери в качестве.

In [14]:
STOPWORDS = nltk.corpus.stopwords.words('english')

Реализуйте функцию для токенизации текста. Выполнять токенизацию можно по-разному, но в данном задании предлагается это делать следующим образом:
1. Привести текст к нижнему регистру
2. Убрать html разметку из текстов (`<br />`, ...)
3. Убрать все символы кроме латинских букв
4. Разбить строку по пробелам
5. Убрать стоп слова

In [15]:
def tokenize(text):
    """
    :param str text: Input text
    :return List[str]: List of words
    """
    # делаем все буквы строчными
    text = text.lower()
    # Найти все подстроки, которые начинаются с <, затем содержат один или более любых символов, кроме >, и заканчиваются >
    text = regex.sub(r'<[^>]+>', '', text)
    # удаляет все символы, кроме срочных букв и пробелов
    text = regex.sub(r'[^a-z\s]', '', text)
    # удаляем стоп-слова
    text = [word for word in text.split() if word not in STOPWORDS]
    return text

In [16]:
tokenize('1. Hello <br />  words!!   I am program!  <br />')

['hello', 'words', 'program']

Теперь мы можем создать словарь, с помощью которого мы будем кодировать токены из текста в числа и наоборот. Для этого мы воспользуемся библиотекой [tokenizers](https://huggingface.co/docs/tokenizers/index)

Токенезация происходит через класс `tokenizer`. Для того чтобы получить `tokenizer` его надо сначала **обучить**, для этого нам необходимо использовать `tokenizers.trainers`. Так как мы будем работать на уровне слов, то выберем `tokenizers.trainers.WordLevelTrainer`.

Для работы с текстами нам необходимо зарезервировать два специальных токена:
1. `<pad>` для токена означающего паддинг
2. `<unk>` для токенов, которые отсутствуют в словаре
3. `<sos>` для токенов, которые обозначают начало последовательности (потребуется во второй части задания)
4. `<eos>` для токенов, которые обозначают конец последовательности (потребуется во второй части задания)

Для начала мы будем разбивать предложение по словам, для этого воспользуемся `trainers.WordLevelTrainer`. Будем рассматривать словарь размером `top_n_words` слов. Подробнее про различных `trainers` можно почитать в [документации](https://huggingface.co/docs/tokenizers/api/trainers), например, из коробки можно использовать BPE.

```python
trainers.WordLevelTrainer(self, /, *args, **kwargs)
Docstring:     
Trainer capable of training a WorldLevel model

Args:
    vocab_size (:obj:`int`, `optional`):
        The size of the final vocabulary, including all tokens and alphabet.

    min_frequency (:obj:`int`, `optional`):
        The minimum frequency a pair should have in order to be merged.

    show_progress (:obj:`bool`, `optional`):
        Whether to show progress bars while training.

    special_tokens (:obj:`List[Union[str, AddedToken]]`):
        A list of special tokens the model should know of.
```

Не забудьте добавить специальные токены. В первой части это токены, отвечающие за паддинг и слова, которых нет в словаре.

In [17]:
# YOUR CODE HERE

trainer = trainers.WordLevelTrainer(
    vocab_size = top_n_words,
    min_frequency = 1,
    show_progress = True,
    special_tokens = ["<unk>", "<pad>"]
)

tokenizer = tokenizers.Tokenizer(
    model = tokenizers.models.WordLevel(unk_token="<unk>")
)

Обучить `tokenizers.Tokenizer` можно **двумя способами**:

- [указать список файлов и обучиться на них](https://huggingface.co/docs/tokenizers/pipeline), бывает полезно, когда датасет нельзя поместить в оперативную память;

- [обучиться из памяти](https://huggingface.co/docs/tokenizers/training_from_memory), то есть хранить датасет в оперативной памяти, бывает полезно при маленьких датасетах как наш.


Чтобы обучиться из файлов нам необходимо задать `tokenizers.normalizers.Normalizer` для нормализации строк (удаление лишних слов и символов), после чего необходимо задать `tokenizers.pre_tokenizers.PreTokenizer` для разделение строки на слова. Подробнее можно посмотреть [в этом примере](https://github.com/huggingface/tokenizers/blob/4383a25787cf366f5e8eaf12643b64f0ba548dc2/bindings/python/examples/custom_components.py). Такая сложность обусловленна тем, что `tokenizers` крайне много использует особенности ООП (объектно-ориентированного программирования).

Мы будем обучаться из памяти, поэтому нам необходимо сделать итератор, который пройдет по всем файлам и токенизирует текст в них их. Подробнее можно почитать [в официальной документации](https://huggingface.co/docs/tokenizers/training_from_memory).


In [18]:
def get_data_iterator():
    for path in ['./aclImdb/test/neg', './aclImdb/test/pos', './aclImdb/train/neg', './aclImdb/train/pos']:
        paths = sorted(list(os.listdir(path)))

        # YOUR CODE HERE
        for file_path in paths:
            text = open(os.path.join(path, file_path), 'r', encoding='utf-8', errors='ignore').read().strip()
            yield tokenize(text)

In [19]:
# именно в этот момент появляется словарь слов (из get_data_iterator)
tokenizer.train_from_iterator(tqdm(get_data_iterator(), total=50_000), trainer=trainer)

100%|██████████| 50000/50000 [00:33<00:00, 1470.64it/s]


Посмотрим на токены с наименьшим *id*. Обратим внимание, что специальные токены имеют наименьшие *id* по-умолчанию.

In [20]:
for i in range(10):
    token = tokenizer.id_to_token(i)
    print(f"ID = {i}, token = {token}")


ID = 0, token = <unk>
ID = 1, token = <pad>
ID = 2, token = movie
ID = 3, token = film
ID = 4, token = one
ID = 5, token = like
ID = 6, token = good
ID = 7, token = even
ID = 8, token = would
ID = 9, token = time


In [21]:
# assert tokenizer.get_vocab_size() == top_n_words

# assert tokenizer.id_to_token(464) == 'complete', f"token = {tokenizer.id_to_token(464)}"
# assert tokenizer.id_to_token(646) == 'typical',  f"token = {tokenizer.id_to_token(646)}"
# assert tokenizer.id_to_token(573) == 'fast',     f"token = {tokenizer.id_to_token(573)}"

**Важно:** При создании итератора мы сортировали файлы, поэтому результат при корретной реализации должен быть детерминированный.

Для кодирования предложений используется метод `encode`, так как мы самостоятельно описали функцию `tokenize`, то установим `is_pretokenized=True`.

In [22]:
result = tokenizer.encode(
    sequence        = tokenize('1. Hello <br /> words!! <br />'),
    is_pretokenized = True
)

print(result)
print("Tokens ids: ", result.ids)

Encoding(num_tokens=2, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Tokens ids:  [4988, 543]


Для декодирования, следует использовать метод `decode`, по умолчанию все специальные токены будут пропущены, то есть все токены `<unk>` будут пропущены.

In [23]:
decode_res = tokenizer.decode(
    ids                 = result.ids,
    skip_special_tokens = True
)
print(f"Decode result: {decode_res}")

Decode result: hello words


Теперь мы готовы создать обёртку-датасет для наших данных.

Необходимо добавить несколько опций, которые понадобятся во второй части задания:
1. Ограничение на максимальную длину текста в токенах. Если текст оказывается длиннее, то последние токены отбрасываются. Иметь ограничение на максимальную длину бывает полезно, так вы имеете гарантии, что во время обучения не засэмплируется очень большой батч, после которого упадет обучение с ошибкой **CUDA error: out of memory**. Кроме того, вы гарантированно знаете на контекстах какой длины обучалась модель, то есть если на валидации вам подасться текст большей длины, то он гарантировано будет отличаться от обучащей выборки.
2. Возможность добавить в специальные токены `<sos>`, `<eos>` в начало и конец токенизированного текста
    
**tips:**
1. В исходных данных рейтинг закодирован в названии файла в виде числа от $1$ до $10$. Для удобства, вычтите $1$, чтобы рейтинг был от $0$ до $9$

In [24]:
class LargeMovieReviewDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_len, pad_sos=False, pad_eos=False):
        """
        :param str data_path: Path to folder with one of the data splits (train or test)
        :param tokenizers.Tokenizer tokenizer: Pretrained tokenizer
        :param int max_len: Maximum length of tokenized text
        :param bool pad_sos: If True, prepend <sos> token
        :param bool pad_eos: If True, append <eos> token
        """
        super().__init__()

        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad_sos = pad_sos
        self.pad_eos = pad_eos

        self.sos_id = tokenizer.token_to_id('<sos>') if pad_sos else None
        self.eos_id = tokenizer.token_to_id('<eos>') if pad_eos else None

        self.negative_paths = sorted([os.path.join(data_path, 'neg', f) for f in os.listdir(os.path.join(data_path, 'neg'))])
        self.positive_paths = sorted([os.path.join(data_path, 'pos', f) for f in os.listdir(os.path.join(data_path, 'pos'))])

        self.labels = [0] * len(self.negative_paths) + [1] * len(self.positive_paths)
        self.paths = self.negative_paths + self.positive_paths

        self.texts = []
        self.tokens = []
        self.ratings = []

        for path in self.paths:
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read().strip()

            res = tokenizer.encode(tokenize(text), is_pretokenized=True)
            token_ids = res.ids

            if self.pad_sos:
                token_ids = [self.sos_id] + token_ids
            if self.pad_eos:
                token_ids.append(self.eos_id)

            token_ids = token_ids[:max_len]

            rating = int(path.split('_')[-1].split('.')[0]) - 1

            self.texts.append(text)
            self.tokens.append(token_ids)
            self.ratings.append(rating)

    def __getitem__(self, idx):
        """
        :param int idx: Index of object in dataset
        :return dict: Dictionary with object data
        """
        return {
            'text': self.texts[idx],
            'label': torch.tensor(self.labels[idx], dtype=torch.long),
            'tokens': torch.tensor(self.tokens[idx], dtype=torch.long),
            'tokens_len': torch.tensor(len(self.tokens[idx]), dtype=torch.long),
            'rating': torch.tensor(self.ratings[idx], dtype=torch.long),
        }

    def __len__(self):
        """Return number of objects in dataset"""
        return len(self.labels)


Создайте датасеты для тестовой и обучающей выборки.

Обратите внимание, что для задачи классификации нам не потребуется дополнять текст с помощью `<sos>`, `<eos>`. Эти токены отвечают за обозначение начала последовательности (**start of sequence**) и её конца (**end of sequence**). При моделировании языка нам будет необходимо уметь понимать где начался и закончился текст. Например, предложения часто начинаются со слова **"однажды"** и крайне редко со слова **"щекотать"**, аналогично про конец предложения.

Не забудьте обрезать длинные тексты, передав параметр `max_length`.

In [25]:
# YOUR CODE HERE
test_dataset = LargeMovieReviewDataset("./aclImdb/test", tokenizer, max_length, pad_sos=False, pad_eos=False)
train_dataset = LargeMovieReviewDataset("./aclImdb/train", tokenizer, max_length, pad_sos=False, pad_eos=False)

Посмотрим, как выглядит объект в датасете:

In [26]:
train_dataset[2]

{'text': "This film lacked something I couldn't put my finger on at first: charisma on the part of the leading actress. This inevitably translated to lack of chemistry when she shared the screen with her leading man. Even the romantic scenes came across as being merely the actors at play. It could very well have been the director who miscalculated what he needed from the actors. I just don't know.<br /><br />But could it have been the screenplay? Just exactly who was the chef in love with? He seemed more enamored of his culinary skills and restaurant, and ultimately of himself and his youthful exploits, than of anybody or anything else. He never convinced me he was in love with the princess.<br /><br />I was disappointed in this movie. But, don't forget it was nominated for an Oscar, so judge for yourself.",
 'label': tensor(0),
 'tokens': tensor([   3, 3001,   52,  286,  162, 3631,   20, 3270,   78,  837,  414, 4941,
            0,  438, 1041,    0,  179,  837,   50,    7,  606,   51,

In [27]:
test_dataset[0]

{'text': "Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.",
 'label': tensor(0),
 'tokens': tensor([ 337,    0, 3162,    2,  127,  993, 1552, 1077,

Теперь нам нужно создать `DataLoader` для наших данных. `DataLoader` умеет из коробки объединять список объектов из датасета в один батч, даже когда датасет возвращает словарь тензоров. Однако, это работает только в случае когда все эти тензоры имеют один и тот же размер во всех батчах. В нашем случае, это не так, так как разные тексты могут иметь разную длину.

Чтобы обойти эту проблему у `DataLoader` есть параметр `collate_fn`, который позволяет задать функцию для объединения списка объектов в один батч.

**tips**
1. Обратите свое внимание на функцию `torch.stack`, она позволяет "застакать" элементы списка в тензор

Чтобы объединить несколько тензоров разной длины в один можно использовать функцию `torch.nn.utils.rnn.pad_sequence`. Такой формат позволит удобно передавать данные в rnn модель.

Обратите внимание на её аргументы:
1. `batch_first` определяет по какой оси "складывать" тензоры. Предпочтительнее использовать `batch_first=False` так как это может упростить выполнение задания в дальнейшем
2. `padding_value` — число, которое будет использоваться в качестве паддинга, чтобы сделать все тензоры одинаковой длины


In [28]:
elemets = [
    torch.tensor([1, 2, 3]),
    torch.tensor([4, 5]),
    torch.tensor([6, 7, 8, 9])
]

out_nbf = torch.nn.utils.rnn.pad_sequence(
    elemets,
    batch_first   = False,
    padding_value = -1
)

out_bf = torch.nn.utils.rnn.pad_sequence(
    elemets,
    batch_first   = True,
    padding_value = -1
)

print(f"batch_first=False")
print(f"Shape = {out_nbf.shape}")
print(out_nbf)

print(f"batch_first=True")
print(f"Shape = {out_bf.shape}")
print(out_bf)

batch_first=False
Shape = torch.Size([4, 3])
tensor([[ 1,  4,  6],
        [ 2,  5,  7],
        [ 3, -1,  8],
        [-1, -1,  9]])
batch_first=True
Shape = torch.Size([3, 4])
tensor([[ 1,  2,  3, -1],
        [ 4,  5, -1, -1],
        [ 6,  7,  8,  9]])


In [29]:
def collate_fn(batch, padding_value, batch_first=False):
    """
    :param List[Dict] batch: List of objects from dataset
    :param int padding_value: Value that will be used to pad tokens
    :param bool batch_first: If True resulting tensor with tokens must have shape [B, T] otherwise [T, B]
    :return dict: Dictionary with all data collated
        {
            'ratings' torch.Tensor(dtype=torch.long): rating of the text for each object in batch
            'labels' torch.Tensor(dtype=torch.long): sentiment of the text for each object in batch

            'texts' List[str]: All texts in one list
            'tokens' torch.Tensor(dtype=torch.long): tensor of tokens ids padded with @padding_value
            'tokens_lens' torch.Tensor(dtype=torch.long): number of tokens for each object in batch
        }
    """
    texts = [item['text'] for item in batch]
    labels = torch.stack([item['label'] for item in batch])
    ratings = torch.stack([item['rating'] for item in batch])
    tokens_lens = torch.stack([item['tokens_len'] for item in batch])

    tokens = torch.nn.utils.rnn.pad_sequence([item['tokens'] for item in batch],
                          batch_first=batch_first,
                          padding_value=padding_value)

    return {
        'texts': texts,
        'labels': labels,
        'ratings': ratings,
        'tokens': tokens,
        'tokens_lens': tokens_lens
    }


Создайте даталоадеры с использованием `collate_fn`.

**tips**:
1. Передать в `collate_fn` правильное значение паддинга можно, например, с помощью `functools.partial`
2. Если вы работаете в Google Colab, то, возможно, вам будет необходимо установить `num_workers=0` во избежание падения ноутбука.
3. Для определения индекса `<pad>` надо использовать `tokenizer.token_to_id('<pad>')`, а не магическую константу 0.

In [30]:
import functools


# Определяем значение паддинга
pad_token_id = tokenizer.token_to_id('<pad>')

# Создаём даталоадеры
collate_fn_partial = functools.partial(collate_fn, padding_value=pad_token_id, batch_first=False)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn_partial, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn_partial, num_workers=0)

Посмотрим на какой-нибудь батч:

In [31]:
batch = next(iter(test_dataloader))
batch.keys(), batch['labels'], batch['ratings'], batch['tokens'], batch['tokens_lens']

(dict_keys(['texts', 'labels', 'ratings', 'tokens', 'tokens_lens']),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([1, 3, 0, 2, 2, 1, 1, 1, 3, 3, 2, 2, 1, 0, 0, 3, 1, 3, 2, 0, 0, 0, 3, 0,
         2, 3, 3, 2, 1, 2, 0, 2]),
 tensor([[ 337,  346,   20,  ...,    0,    7,    0],
         [   0, 1996,  612,  ..., 1139,    0,    0],
         [3162,  122,    0,  ..., 2471, 1436,   53],
         ...,
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1]]),
 tensor([ 74, 128, 108, 168, 137,  52,  75,  74,  72,  98,  59, 143, 134,  52,
         104, 111,  67, 116, 189,  47,  35,  96, 200, 200, 136, 111, 105, 200,
         200, 144,  75,  82]))

# `Часть 1. Классификация текстов (4 балла)`

В этой части вы обучите классификатор текстов на основе рекуррентной нейронной сети. Выше мы уже создали удобные класс-обёртки для работы с данными. Теперь мы соберем модель для решения задачи классификации. Вам предлагается решить задачу предсказания **рейтинга фильма** (классы 0 до 9), то есть мы решаем задачу многоклассовой классификации.

## `Сборка и обучение RNN в pytorch (1 балл)`

Создадим переменные для device-agnostic кода:

In [32]:
dtype, device, cuda_device_id = torch.float32, None, 0
os.environ["CUDA_VISIBLE_DEVICES"] = '{0}'.format(str(cuda_device_id) if cuda_device_id is not None else '')
if cuda_device_id is not None and torch.cuda.is_available():
    device = 'cuda:{0:d}'.format(0)
else:
    device = torch.device('cpu')
print(f'Using device: {device}, dtype: {dtype}')

Using device: cuda:0, dtype: torch.float32


Наша нейросеть будет обрабатывать входную последовательность по словам (word level). Мы будем использовать простую и стандартную рекуррентную архитектуру для классификации:
1. Слой представлений, превращающий id токена в вектор-эмбеддинг этого слова
2. Слой LSTM
3. Полносвязный слой, предсказывающий выход по последнему скрытому состоянию

Ниже дан код для сборки и обучения нашей нейросети.

Допишите класс-обёртку над LSTM для задачи классификации.
**Не используйте циклы.**

**Для каждого тензора в функции `forward` подпишите в комментарии его размеры**

In [33]:
class RNNClassifier(torch.nn.Module):
    def __init__(
        self, embedding_dim, hidden_dim, output_size, tokenizer,
        rec_layer=torch.nn.LSTM, dropout=None, device=None, **kwargs
    ):
        super().__init__()

        self.device = device
        self.dropout = dropout
        self.tokenizer = tokenizer
        self.hidden_dim = hidden_dim
        self.output_size = output_size
        self.embedding_dim = embedding_dim

        self.word_embeddings = torch.nn.Embedding(num_embeddings=tokenizer.get_vocab_size(),
                                                  embedding_dim=embedding_dim,
                                                  padding_idx=tokenizer.token_to_id('<pad>'))

        self.rnn = rec_layer(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=False,  # Формат входных данных (T, B, F)
            dropout=dropout if dropout is not None else 0.0,
            **kwargs
        )

        self.output = torch.nn.Linear(hidden_dim, output_size)

    def forward(self, tokens, tokens_lens):
        """
        :param torch.Tensor(dtype=torch.long) tokens: Batch of texts represented with tokens.
        :param torch.Tensor(dtype=torch.long) tokens_lens: Number of non-padding tokens for each object in batch.
        :return torch.Tensor(dtype=torch.long): Vector representation for each sequence in batch
        """
        embeds = self.word_embeddings(tokens)  # (T, B, embedding_dim)

        rnn_out, _ = self.rnn(embeds)  #  (T, B, hidden_dim)

        last_hidden_states = rnn_out[tokens_lens - 1, torch.arange(tokens_lens.shape[0])]  # (B, hidden_dim)

        logits = self.output(last_hidden_states)  # (B, output_size)

        return logits

[Исходный код LSTM](http://pytorch.org/docs/master/_modules/torch/nn/modules/rnn.html#LSTM)

Допишите функции для обучения и оценки модели:

**tip:**
1. В функции `evaluate` при подсчёте метрик учитывайте, что батчи могут иметь разный размер. (в частности последний батч)

**Важно:** Мы предсказываем `rating`, не `label`.

In [34]:
import wandb

def train_epoch(dataloader, model, loss_fn, optimizer, device):
    model.train()

    total_loss = 0.0
    for idx, data in enumerate(dataloader):
        # 1. Take data from batch
        tokens = data['tokens'].to(device)
        true_rating = data['ratings'].to(device)
        tokens_lens = data['tokens_lens'].cpu()

        # 2. Perform forward pass
        pred_rating = model(tokens, tokens_lens)

        # 3. Evaluate loss
        loss = loss_fn(pred_rating, true_rating)
        total_loss += loss.item() * len(true_rating)  # умножаем на размер батча, чтобы учесть, если вдруг последний батч будет меньше

        # 4. Make optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


def evaluate(dataloader, model, loss_fn, device, prefix):
    model.eval()

    total_loss = 0.0
    total_accuracy = 0.0
    with torch.no_grad():
        for idx, data in enumerate(dataloader):
            # 1. Take data from batch
            tokens = data['tokens'].to(device)
            true_rating = data['ratings'].to(device)
            tokens_lens = data['tokens_lens'].cpu()

            # 2. Perform forward pass
            pred_rating = model(tokens, tokens_lens)  # конкретно здесь просто логиты, результат последнего полносвязного слоя (nn.Linear)

            # 3. Evaluate loss
            loss = loss_fn(pred_rating, true_rating) # считаем лосс на логитах, тк в CrossEtrophy softmax уже включен, не надо применять его дважды
            total_loss += loss.item() * len(true_rating)

            # 4. Evaluate accuracy
            prediction = pred_rating.argmax(dim=1) # берем аргмакс, чтобы извлечь сами классы
            total_accuracy += (prediction == true_rating).sum().item()

    loss = total_loss / len(dataloader.dataset)
    accuracy = total_accuracy / len(dataloader.dataset)

    wandb.log({
        f"{prefix}_loss": loss,
        f"{prefix}_accuracy": accuracy
    })

    return loss, accuracy


def train(train_loader, test_loader, model, loss_fn, optimizer, device, num_epochs):
    test_losses = []
    train_losses = []
    test_accuracies = []
    train_accuracies = []

    start_time = time.time()
    for epoch in range(num_epochs):
        epoch_start = time.time()

        train_epoch(train_loader, model, loss_fn, optimizer, device)

        train_loss, train_acc = evaluate(train_loader, model, loss_fn, device, prefix="train")
        train_accuracies.append(train_acc)
        train_losses.append(train_loss)

        test_loss, test_acc = evaluate(test_loader, model, loss_fn, device, prefix="test")
        test_accuracies.append(test_acc)
        test_losses.append(test_loss)

        epoch_time = time.time() - epoch_start
        wandb.log({"epoch_time": epoch_time})
        print(
            'Epoch: {0:d}/{1:d}. Loss (Train/Test): {2:.3f}/{3:.3f}. Accuracy (Train/Test): {4:.3f}/{5:.3f}. Time: {6:.3f}'.format(
                epoch + 1, num_epochs, train_losses[-1], test_losses[-1], train_accuracies[-1], test_accuracies[-1], epoch_time
            )
        )

    total_time = time.time() - start_time
    wandb.log({"total_training_time": total_time})
    return train_losses, train_accuracies, test_losses, test_accuracies


Создадим модель:

In [35]:
set_global_seed(42)

model = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, tokenizer=tokenizer,
    rec_layer=torch.nn.LSTM, dropout=None
).to(device)

check_numel(model, 244234, 0)

Создадим класс для подсчёта функции потерь и оптимизатор:

In [36]:
loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Попробуем обучить модель:

In [37]:
import wandb


wandb.login()
wandb.init(project="DL_rnn", name="rnn_baseline", config={
    "epochs": num_epochs,
    "batch_size": train_dataloader.batch_size,
    "lr": optimizer.param_groups[0]["lr"],
    "embedding_dim": model.embedding_dim,
    "hidden_dim": model.hidden_dim,
    "dropout": model.dropout,
})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: keiiino (keiiino_) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [38]:
import time

start = time.time()
train_losses_pure, train_accuracies_pure, test_losses_pure, test_accuracies_pure = train(
    train_dataloader, test_dataloader, model, loss_fn, optimizer, device, num_epochs)
end = time.time()

print(f"Модель обучалась {int((end - start) // 60)} мин {int((end - start) % 60)} сек")

Epoch: 1/15. Loss (Train/Test): 1.966/1.976. Accuracy (Train/Test): 0.279/0.276. Time: 9.716
Epoch: 2/15. Loss (Train/Test): 1.640/1.686. Accuracy (Train/Test): 0.369/0.355. Time: 7.855
Epoch: 3/15. Loss (Train/Test): 1.534/1.626. Accuracy (Train/Test): 0.398/0.374. Time: 8.567
Epoch: 4/15. Loss (Train/Test): 1.475/1.611. Accuracy (Train/Test): 0.423/0.387. Time: 8.630
Epoch: 5/15. Loss (Train/Test): 1.367/1.555. Accuracy (Train/Test): 0.466/0.394. Time: 7.767
Epoch: 6/15. Loss (Train/Test): 1.331/1.610. Accuracy (Train/Test): 0.475/0.395. Time: 8.608
Epoch: 7/15. Loss (Train/Test): 1.218/1.589. Accuracy (Train/Test): 0.526/0.395. Time: 8.737
Epoch: 8/15. Loss (Train/Test): 1.127/1.641. Accuracy (Train/Test): 0.560/0.392. Time: 7.817
Epoch: 9/15. Loss (Train/Test): 1.049/1.683. Accuracy (Train/Test): 0.597/0.388. Time: 8.553
Epoch: 10/15. Loss (Train/Test): 0.981/1.752. Accuracy (Train/Test): 0.633/0.367. Time: 8.535
Epoch: 11/15. Loss (Train/Test): 0.840/1.860. Accuracy (Train/Test): 

**Сохраните все метрики и время работы модели. Это потребуется в конце первой части для построения графиков обучения и сравнения времени работы для всех моделей в этой секции**

**Возможно стоит сохранить результаты в виде файлов и скачать их**

**Обратите внимание, что надо сохранить и время**

Нерегуляризованные LSTM часто быстро переобучаются (и мы это видим по точности на контроле). Чтобы с этим бороться, часто используют *L2-регуляризацию* и *дропаут*.
Однако способов накладывать дропаут на рекуррентный слой достаточно много, и далеко не все хорошо работают. По [ссылке](https://medium.com/@bingobee01/a-review-of-dropout-as-applied-to-rnns-72e79ecd5b7b) доступен хороший обзор дропаутов для RNN.

Мы реализуем два варианта DropOut для RNN (и третий дополнительно). Заодно увидим, что для реализации различных усовершенствований рекуррентной архитектуры приходится "вскрывать" слой до различной "глубины".

## `Реализация дропаута по статье Гала и Гарамани. Variational Dropout (1 балл)`

Начнем с дропаута, описанного в [статье Гала и Гарамани](https://arxiv.org/abs/1512.05287).
Для этого нам потребуется перейти от использования слоя `torch.nn.LSTM`, полностью скрывающего от нас рекуррентную логику, к использованию слоя `torch.nn.LSTMCell`, обрабатывающего лишь один временной шаг нашей последовательности (а всю логику вокруг придется реализовать самостоятельно).

Для начала напишем функцию, которая позволит нам получать $h_0$ и $c_0$, которые мы будем использовать в качестве инициализаций.

**tips:**

1. Используйте some_existing_tensor, как тензор в котором содержится информация о типе данных, девайсе целевого тензора. Обратите внимание на функцию [new_ones](https://pytorch.org/docs/stable/generated/torch.Tensor.new_ones.html).

In [39]:
def init_h0_c0(num_objects, hidden_size, some_existing_tensor):
    """
    return h0 and c0, use some_existing_tensor.new_zeros() to gen them
    h0 shape: num_objects x hidden_size
    c0 shape: num_objects x hidden_size
    """
    h0 = some_existing_tensor.new_ones((num_objects, hidden_size))
    c0 = some_existing_tensor.new_ones((num_objects, hidden_size))
    return h0, c0

Допишите класс `RNNLayer`. При `dropout=0` ваш класс должен работать как обычный слой LSTM, а при `dropout > 0` накладывать бинарную маску на входной и скрытый вектор на каждом временном шаге, причем эта маска должна быть одинаковой во все моменты времени.

Дропаут Гала и Гарамани в виде формул (m обозначает маску дропаута):

$$
h_{t-1} = h_{t-1}*m_h, \, x_t = x_t * m_x
$$

Далее обычный шаг рекуррентной архитектуры, например, LSTM:

$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o)
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad
g = tanh(h_{t-1} W^g + x_t U^g+b_g)
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t)
$$

**Важно**: Мы считаем, что объекты в батче независимы, то есть маски для них должны быть разные.

**tips:**

1. Для получения бернулливской случайной величины достаточно вызвать `.bernoulli()` от массива содержащего вероятности.


In [40]:
def gen_dropout_mask(input_size, hidden_size, is_training, p, some_existing_tensor):
    """
    is_training: if True, gen masks from Bernoulli
                 if False, gen masks consisting of (1-p)

    return dropout masks of size input_size, hidden_size if p is not None
    return one masks if p is None
    """
    if p is not None and is_training:
        mask = some_existing_tensor.new_empty(input_size, hidden_size).bernoulli_(1 - p)
    elif p is not None and not is_training:
        mask = some_existing_tensor.new_full((input_size, hidden_size), 1 - p)
    else:
        mask = some_existing_tensor.new_ones((input_size, hidden_size))
    return mask

In [41]:
set_global_seed(42)
p = 0.12
m = gen_dropout_mask(100, 120, is_training=True, p=p, some_existing_tensor=torch.tensor(1.))


print(f'm.mean(): {m.mean():0.4f}')
assert m.shape == (100, 120)
assert (1 - p) - 0.005 <= m.mean() <= (1 - p) + 0.005

m.mean(): 0.8825


Допишите класс-обёртку над `LSTMCell` для реализации Variational Dropout. **Используйте только цикл по времени**

**Для каждого тензора в функции `forward` подпишите в комментарии его размеры**

In [81]:
class RNNLayer(torch.nn.Module):
    def __init__(self, input_size, hidden_size, dropout=None, **kwargs):
        super().__init__()
        self.dropout = dropout
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.rnn_cell = torch.nn.LSTMCell(self.input_size, self.hidden_size)

    def forward(self, x):
        """
        x: (T, B, D) обычный тензор эмбеддингов
        """
        is_training = self.training
        batch_size = x.shape[1]

        h_t, c_t = init_h0_c0(batch_size, self.hidden_size, x[0])

        mask_x = gen_dropout_mask(x.shape[1], self.input_size, is_training=is_training, p=self.dropout, some_existing_tensor=x)
        mask_h = gen_dropout_mask(x.shape[1], self.hidden_size, is_training=is_training, p=self.dropout, some_existing_tensor=h_t)

        outputs = []
        for t in range(x.shape[0]):
            x_t = x[t] * mask_x if self.dropout is not None else x[t]
            h_t, c_t = self.rnn_cell(x_t, (h_t, c_t))
            h_t = h_t * mask_h if self.dropout is not None else h_t
            outputs.append(h_t)

        outputs = torch.stack(outputs)
        return outputs, (h_t, c_t)


In [78]:
layer = RNNLayer(32, 64, dropout=None)
dummy_input = torch.ones((10, 16, 32))

out = layer(dummy_input)

assert out[0].shape == (10, 16, 64)
assert out[1][0].shape == (16, 64)
assert out[1][1].shape == (16, 64)

Протестируйте реализованную модель с выключенным дропаутом (слой `RNNLayer` надо передать в `RNNClassifier` в качестве `rec_layer`). Замерьте время обучения. Сильно ли оно увеличилось по сравнению с `torch.nn.LSTM` (LSTM "из коробки")?

**Сохраните все метрики и время работы модели. Это потребуется в конце первой части для построения графиков обучения и сравнения времени работы для всех моделей в этой секции**

**Возможно стоит сохранить результаты в виде файлов и скачать их**

In [82]:
set_global_seed(42)

model_no_dropout = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, tokenizer=tokenizer,
    rec_layer=RNNLayer, dropout=None
).to(device)


wandb.init(project="DL_rnn", name="rnn_custom_dropout_none", config={
    "epochs": num_epochs,
    "batch_size": train_dataloader.batch_size,
    "lr": optimizer.param_groups[0]["lr"],
    "embedding_dim": model_no_dropout.embedding_dim,
    "hidden_dim": model_no_dropout.hidden_dim,
    "dropout": model_no_dropout.dropout,
})

epoch_time,█▁
test_accuracy,▁▁
test_loss,▁▁
train_accuracy,▁▁
train_loss,▁▁
epoch_time,203.48275
test_accuracy,0.12432
test_loss,2.29813
train_accuracy,0.1188
train_loss,2.29786


In [83]:
start = time.time()
train_losses_custom_no_dropout, train_accuracies_custom_no_dropout, test_losses_custom_no_dropout, test_accuracies_custom_no_dropout = train(
    train_dataloader, test_dataloader, model_no_dropout, loss_fn, optimizer, device, num_epochs)
end = time.time()

print(f"Модель обучалась {int((end - start) // 60)} мин {int((end - start) % 60)} сек")

Epoch: 1/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 127.845
Epoch: 2/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 126.118
Epoch: 3/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 126.520


KeyboardInterrupt: 

Протестируйте полученную модель с `dropout=0.25`, вновь замерив время обучения. Получилось ли побороть переобучение? Сильно ли дольше обучается данная модель по сравнению с предыдущей? (доп. время тратится на генерацию масок дропаута).

In [46]:
set_global_seed(42)

model_dropout_025 = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, tokenizer=tokenizer,
    rec_layer=RNNLayer, dropout=0.25
).to(device)

wandb.init(project="DL_rnn", name="rnn_custom_dropout_0_25", config={
    "epochs": num_epochs,
    "batch_size": train_dataloader.batch_size,
    "lr": optimizer.param_groups[0]["lr"],
    "embedding_dim": model_dropout_025.embedding_dim,
    "hidden_dim": model_dropout_025.hidden_dim,
    "dropout": model_dropout_025.dropout,
})

start = time.time()
train_losses_custom_no_dropout, train_accuracies_custom_no_dropout, test_losses_custom_no_dropout, test_accuracies_custom_no_dropout = train(
    train_dataloader, test_dataloader, model_dropout_025, loss_fn, optimizer, device, num_epochs)
end = time.time()

print(f"Модель обучалась {int((end - start) // 60)} мин {int((end - start) % 60)} сек")

epoch_time,█▁
test_accuracy,▁▁
test_loss,▁▁
train_accuracy,▁▁
train_loss,▁▁
epoch_time,107.49821
test_accuracy,0.10376
test_loss,2.30584
train_accuracy,0.10756
train_loss,2.30504


Epoch: 1/15. Loss (Train/Test): 2.307/2.308. Accuracy (Train/Test): 0.109/0.104. Time: 109.878
Epoch: 2/15. Loss (Train/Test): 2.307/2.308. Accuracy (Train/Test): 0.109/0.104. Time: 109.753
Epoch: 3/15. Loss (Train/Test): 2.307/2.308. Accuracy (Train/Test): 0.109/0.104. Time: 108.909


KeyboardInterrupt: 

## `Реализация дропаута по статье Гала и Гарамани. Дубль 2 (1 балл)`

<начало взлома pytorch>

При разворачивании цикла по времени средствами python обучение рекуррентной нейросети сильно замедляется. Однако для реализации дропаута Гала и Гарамани необязательно явно задавать в коде умножение нейронов на маски. Можно схитрить и обойтись использованием слоя `torch.nn.LSTM`: перед вызовом `forward` слоя `torch.nn.LSTM` подменять его веса на веса, домноженные **по строкам** на маски. А обучаемые веса хранить отдельно. Именно так этот дропаут реализован в библиотеке `fastai`, код из которой использован в ячейке ниже.

Благодаря такому подходу мы используем быстрый код слоя `torch.nn.LSTM`, который гарантировано написан хорошо и правильно, но подменяем веса так, чтобы получить необходимый эффект.

Для начала посмотрим на стандартный слой `nn.LSTM` и вспомним дропаут Гала и Гарамани (m обозначает маску дропаута):

$$
h_{t-1} = h_{t-1}*m_h, \, x_t = x_t * m_x
$$

Далее обычный шаг рекуррентной архитектуры, например, LSTM:

$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o)
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad
g = tanh(h_{t-1} W^g + x_t U^g+b_g)
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t)
$$

Сначал посмотрим на параметры слоя `nn.LSTM`.

In [47]:
lstm = torch.nn.LSTM(embedding_dim, hidden_dim, num_layers=2)

for tag, p in lstm.named_parameters():
    print(f"{tag:<12}: {p.shape}")

weight_ih_l0: torch.Size([512, 32])
weight_hh_l0: torch.Size([512, 128])
bias_ih_l0  : torch.Size([512])
bias_hh_l0  : torch.Size([512])
weight_ih_l1: torch.Size([512, 128])
weight_hh_l1: torch.Size([512, 128])
bias_ih_l1  : torch.Size([512])
bias_hh_l1  : torch.Size([512])


Подробнее про каждый отдельный модуль можно почитать на [официальной странице](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html), важно, что умножая каждую строку весов **weight\_\*\*\_\*\*** на маску, мы получим те же вычисления, что и при умножении $h_{t-1}$ и $x_t$ на маску. Для проверки этого факта рассмотрим игрушечный пример:

In [48]:
set_global_seed(42)

W = torch.randn((hidden_dim * 4, hidden_dim))
x = torch.arange(hidden_dim).float()
mask = (torch.arange(hidden_dim) % 2 == 0).long()

res1 = W @ (x * mask)

res2 = (W * mask[None, :]) @ x

torch.isclose(res1, res2).all()

tensor(True)

Реализуйте слой, который с помощью подмены весов реализует  дропаут Гала и Гарамани, в виде обертки над `torch.nn.LSTM`. Допишите класс:

In [49]:
import warnings

In [75]:
class FastRNNLayer(torch.nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.0, layers_dropout=0.0, num_layers=1, **kwargs):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.num_layers = num_layers

        self.dropout = dropout
        self.layers_dropout = layers_dropout
        self.module = torch.nn.LSTM(input_size, hidden_size, dropout=layers_dropout, num_layers=num_layers)

        self.layer_names = []
        for layer_n in range(self.num_layers):
            self.layer_names += [f'weight_hh_l{layer_n}', f'weight_ih_l{layer_n}']

        for layer in self.layer_names:
            # Get torch.nn.Parameter with weights from torch.nn.LSTM instance
            w = getattr(self.module, layer)

            # Remove it from model
            delattr(self.module, layer)

            # And create new torch.nn.Parameter with the same data but different name
            self.register_parameter(f'{layer}_raw', torch.nn.Parameter(w.data))

            # Note. In torch.nn.LSTM.forward parameter with name `layer` will be used
            #     so we must initialize it using `layer_raw` before forward pass

    def _setweights(self, x):
        """
            Apply dropout to the raw weights.
        """
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')

            dropout_mask = gen_dropout_mask(raw_w.shape[0], raw_w.shape[1], is_training=self.training, p=self.dropout, some_existing_tensor=x)

            masked_raw_w = raw_w * dropout_mask

            setattr(self.module, layer, masked_raw_w)

    def forward(self, x, h_c=None):
        """
        :param x: tensor containing the features of the input sequence.
        :param Optional[Tuple[torch.Tensor, torch.Tensor]] h_c: initial hidden state and initial cell state
        """
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")

            self._setweights(x)

            if h_c is not None:
                output, (h, c) = self.module(x, h_c)
            else:
                output, (h, c) = self.module(x)
            return output, (h, c)

    def reset(self):
        if hasattr(self.module, 'reset'):
            self.module.reset()

In [76]:
layer = FastRNNLayer(32, 64, dropout=None)
dummy_input = torch.ones((10, 16, 32))

out = layer(dummy_input)

assert out[0].shape == (10, 16, 64)
assert out[1][0].shape == (1, 16, 64)
assert out[1][1].shape == (1, 16, 64)

Протестируйте реализованную модель с выключенным дропаутом (слой `FastRNNLayer` надо передать в `RNNClassifier` в качестве `rec_layer`). Замерьте время обучения. Убедитесь, что модель выдаёт такое же качество, как и оригинальная реализация LSTM.

**Сохраните все метрики и время работы модели. Это потребуется в конце первой части для построения графиков обучения и сравнения времени работы для всех моделей в этой секции**

**Возможно стоит сохранить результаты в виде файлов и скачать их**

In [52]:
set_global_seed(42)

model_fastdropout_none = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, tokenizer=tokenizer,
    rec_layer=FastRNNLayer, dropout=None
).to(device)

wandb.init(project="DL_rnn", name="rnn_custom_fast_dropout_none", config={
    "epochs": num_epochs,
    "batch_size": train_dataloader.batch_size,
    "lr": optimizer.param_groups[0]["lr"],
    "embedding_dim": model_fastdropout_none.embedding_dim,
    "hidden_dim": model_fastdropout_none.hidden_dim,
    "dropout": model_fastdropout_none.dropout,
})

start = time.time()
train_losses_custom_no_dropout, train_accuracies_custom_no_dropout, test_losses_custom_no_dropout, test_accuracies_custom_no_dropout = train(
    train_dataloader, test_dataloader, model_fastdropout_none, loss_fn, optimizer, device, num_epochs)
end = time.time()

print(f"Модель обучалась {int((end - start) // 60)} мин {int((end - start) % 60)} сек")

epoch_time,█▇▁
test_accuracy,▁▁▁
test_loss,▁▁▁
train_accuracy,▁▁▁
train_loss,▁▁▁
epoch_time,108.90887
test_accuracy,0.10428
test_loss,2.30827
train_accuracy,0.1088
train_loss,2.30747


Epoch: 1/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 10.055
Epoch: 2/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 9.118
Epoch: 3/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 9.689
Epoch: 4/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 9.968
Epoch: 5/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 9.891
Epoch: 6/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 9.337
Epoch: 7/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 9.418
Epoch: 8/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 9.913
Epoch: 9/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 9.954
Epoch: 10/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test): 0.108/0.104. Time: 9.550
Epoch: 11/15. Loss (Train/Test): 2.305/2.306. Accuracy (Train/Test):

Протестируйте полученный слой (вновь подставив его в `RNNClassifier` в качестве `rec_layer`) с `dropout=0.25`. Сравните время обучения с предыдущими моделями. Проследите, чтобы качество получилось такое же, как при первой реализации этого дропаута.

In [53]:
set_global_seed(42)

model_fastdropout_025 = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, tokenizer=tokenizer,
    rec_layer=FastRNNLayer, dropout=0.25
).to(device)

wandb.init(project="DL_rnn", name="rnn_custom_fast_dropout_025", config={
    "epochs": num_epochs,
    "batch_size": train_dataloader.batch_size,
    "lr": optimizer.param_groups[0]["lr"],
    "embedding_dim": model_fastdropout_025.embedding_dim,
    "hidden_dim": model_fastdropout_025.hidden_dim,
    "dropout": model_fastdropout_025.dropout,
})

start = time.time()
train_losses_custom_no_dropout, train_accuracies_custom_no_dropout, test_losses_custom_no_dropout, test_accuracies_custom_no_dropout = train(
    train_dataloader, test_dataloader, model_fastdropout_025, loss_fn, optimizer, device, num_epochs)
end = time.time()

print(f"Модель обучалась {int((end - start) // 60)} мин {int((end - start) % 60)} сек")

epoch_time,█▁▅▇▇▃▄▇▇▄▂▆█▆▁
test_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_training_time,▁
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_time,9.05892
test_accuracy,0.10376
test_loss,2.30585
total_training_time,144.77396
train_accuracy,0.10756


Epoch: 1/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.421
Epoch: 2/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.898
Epoch: 3/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 10.016
Epoch: 4/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.730
Epoch: 5/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.102
Epoch: 6/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.729
Epoch: 7/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 10.063
Epoch: 8/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.792
Epoch: 9/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 8.922
Epoch: 10/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.708
Epoch: 11/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test)

</конец взлома pytorch>

## `Реализация дропаута по статье Семениуты и др. (1 балл)`

Перейдем к реализации дропаута для LSTM по статье [Semeniuta et al](http://www.aclweb.org/anthology/C16-1165).

Этот метод применения дропаута не менее популярен, чем предыдущий. Его особенность состоит в том, что он придуман специально для гейтовых архитектур. В контексте LSTM этот дропаут накладывается только на информационный поток ($m_h$ — маска дропаута):
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o)
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad
g = tanh(h_{t-1} W^g + x_t U^g+b_g)
$$
$$
c_t = f \odot c_{t-1} +  i \odot g \odot {\bf m_h} \quad
h_t =  o \odot tanh(c_t)
$$
**На входы $x_t$ маска накладывается как в предыдущем дропауте.** Впрочем, на входы маску можно наложить вообще до вызова рекуррентного слоя.

Согласно статье, маска дропаута может быть как одинаковая, так и разная для всех моментов времени. Мы сделаем одинаковую для всех моментов времени.

Для реализации этого дропаута можно:
1. самостоятельно реализовать LSTM (интерфейса LSTMCell не хватит)
2. снова воспользоваться трюком с установкой весов (но тут мы опираемся на свойство $tanh(0)=0$, к тому же, трюк в данном случае выглядит менее тривиально, чем с дропаутом Гала).

**Внимание:** Раньше мы реализовывали через LSTMCell и модель работала долго, теперь при рукописном варианте будет работать еще дольше!

Предлагается реализовать дропаут по сценарию 1. Допишите класс:

**Для каждого тензора в функции `forward` подпишите в комментарии его размеры**

In [59]:
class HandmadeLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.0, **kwargs):
        super().__init__()

        self.dropout = dropout
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.input_weights = torch.nn.Linear(input_size, 4 * hidden_size)
        self.hidden_weights = torch.nn.Linear(hidden_size, 4 * hidden_size)

        self.reset_params()

    def reset_params(self):
        """
        Initialization as in Pytorch.
        Do not forget to call this method!
        https://pytorch.org/docs/stable/_modules/torch/nn/modules/rnn.html#LSTM
        """
        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.parameters():
            torch.nn.init.uniform_(weight, -stdv, stdv)

    def forward(self, x):
        """
        :param x: (T, B, D) — входная последовательность
        :return: (T, B, H), (h_n, c_n)
        """
        T, B, _ = x.shape
        device = x.device

        # (B, H)
        h_t, c_t = init_h0_c0(B, self.hidden_size, x)

        outputs = []

        # маска дропаут для g, одинаковая на всех шагах
        mask_h = gen_dropout_mask(B, self.hidden_size, self.training, self.dropout, x)

        for t in range(T):
            x_t = x[t]  # (B, D)
            gates = self.input_weights(x_t) + self.hidden_weights(h_t)  # (B, 4H)
            i, f, o, g = gates.chunk(4, dim=1)  # Каждое (B, H)

            i = torch.sigmoid(i)
            f = torch.sigmoid(f)
            o = torch.sigmoid(o)
            g = torch.tanh(g)

            # Применяем маску только к g
            g = g * mask_h

            c_t = f * c_t + i * g      # (B, H)
            h_t = o * torch.tanh(c_t)  # (B, H)

            outputs.append(h_t.unsqueeze(0))  # (1, B, H)

        outputs = torch.cat(outputs, dim=0)  # (T, B, H)
        return outputs, (h_t, c_t)

In [60]:
layer = HandmadeLSTM(32, 64, dropout=None)
dummy_input = torch.ones((10, 16, 32))

out = layer(dummy_input)

assert out[0].shape == (10, 16, 64)
assert out[1][0].shape == (16, 64)
assert out[1][1].shape == (16, 64)

Протестируйте вашу реализацию без дропаута (проконтролируйте качество и сравните время обучения с временем обучения `torch.nn.LSTM` и `RNNLayer`), а также с `dropout=0.25`. Сравните качество модели с таким дропаутом с качеством модели с дропаутом Гала и Гарамани.

**Сохраните все метрики и время работы модели. Это потребуется в конце первой части для построения графиков обучения и сравнения времени работы для всех моделей в этой секции**

**Возможно стоит сохранить результаты в виде файлов и скачать их**

In [61]:
set_global_seed(42)

model_handmade_no_dropout = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, tokenizer=tokenizer,
    rec_layer=HandmadeLSTM, dropout=None
).to(device)

wandb.init(project="DL_rnn", name="rnn_handmade_dropout_none", config={
    "epochs": num_epochs,
    "batch_size": train_dataloader.batch_size,
    "lr": optimizer.param_groups[0]["lr"],
    "embedding_dim": model_handmade_no_dropout.embedding_dim,
    "hidden_dim": model_handmade_no_dropout.hidden_dim,
    "dropout": model_handmade_no_dropout.dropout,
})

start = time.time()
train_losses_custom_no_dropout, train_accuracies_custom_no_dropout, test_losses_custom_no_dropout, test_accuracies_custom_no_dropout = train(
    train_dataloader, test_dataloader, model_handmade_no_dropout, loss_fn, optimizer, device, num_epochs)
end = time.time()

print(f"Модель обучалась {int((end - start) // 60)} мин {int((end - start) % 60)} сек")

epoch_time,▁▇█▇▂▅██▃▄██▃
test_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_time,9.28879
test_accuracy,0.10492
test_loss,2.30713
train_accuracy,0.10868
train_loss,2.30631


Epoch: 1/15. Loss (Train/Test): 2.298/2.298. Accuracy (Train/Test): 0.119/0.124. Time: 205.258
Epoch: 2/15. Loss (Train/Test): 2.298/2.298. Accuracy (Train/Test): 0.119/0.124. Time: 203.483


KeyboardInterrupt: 

In [57]:
set_global_seed(42)

model_handmade_dropout_025 = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, tokenizer=tokenizer,
    rec_layer=FastRNNLayer, dropout=0.25
).to(device)

wandb.init(project="DL_rnn", name="rnn_handmade_dropout_025", config={
    "epochs": num_epochs,
    "batch_size": train_dataloader.batch_size,
    "lr": optimizer.param_groups[0]["lr"],
    "embedding_dim": model_handmade_dropout_025.embedding_dim,
    "hidden_dim": model_handmade_dropout_025.hidden_dim,
    "dropout": model_handmade_dropout_025.dropout,
})

start = time.time()
train_losses_custom_no_dropout, train_accuracies_custom_no_dropout, test_losses_custom_no_dropout, test_accuracies_custom_no_dropout = train(
    train_dataloader, test_dataloader, model_handmade_dropout_025, loss_fn, optimizer, device, num_epochs)
end = time.time()

print(f"Модель обучалась {int((end - start) // 60)} мин {int((end - start) % 60)} сек")

epoch_time,█▆▁▅▇▆▂▅▇▆▂▄▇▆▃
test_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_training_time,▁
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_time,9.34772
test_accuracy,0.10492
test_loss,2.30713
total_training_time,144.81983
train_accuracy,0.10868


Epoch: 1/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 8.959
Epoch: 2/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.801
Epoch: 3/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.955
Epoch: 4/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.800
Epoch: 5/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.062
Epoch: 6/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.593
Epoch: 7/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.916
Epoch: 8/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.920
Epoch: 9/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.205
Epoch: 10/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 0.109/0.105. Time: 9.370
Epoch: 11/15. Loss (Train/Test): 2.306/2.307. Accuracy (Train/Test): 

KeyboardInterrupt: 

## `Сравнение всех предложенных моделей (1 балл)`

Используя замеры времени заполните табличку с временем работы четырёх реализованных моделей в следующей ячейке:

| torch.nn.LSTM | RNNLayer | FastRNNLayer | HandmadeLSTM |
|---------------|----------|--------------|--------------|
| 2m 35s        | 14m 16s  | 2m 41s       | 31m 44s      |

In [62]:
import matplotlib.pyplot as plt

Крайне желательно рисовать графики в векторном формате.

Если по каким-то причинам, отрисовка не будет работать, закомментируйте следующую ячейку.

In [63]:
%matplotlib inline

import matplotlib_inline
from IPython.display import set_matplotlib_formats

matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'svg')

Нарисуйте два графика — функция потерь и качество на обучающей и тестовой выборке для всех 7 моделей обученных выше.

In [69]:
import matplotlib.pyplot as plt
import wandb

# Названия моделей
labels = [
    "baseline",
    "custom_dropout=none",
    "custom_dropout=0.25",
    "fast_dropout=none",
    "fast_dropout=0.25",
    "handmade_dropout=none",
    # "handmade_dropout=0.25"
]

# Пути к runs
# Make sure these paths are correct and the runs are accessible.
# If you are using a different entity or project, replace 'Keiiino/DL_rnn' accordingly.
# Also, confirm that the run names are accurate.
run_paths = [
    "Keiiino/DL_rnn/rnn_baseline",  # Example: username/project_name/run_id
    "Keiiino/DL_rnn/rnn_custom_dropout_none",
    "Keiiino/DL_rnn/rnn_custom_dropout_0_25",
    "Keiiino/DL_rnn/rnn_custom_fast_dropout_none",
    "Keiiino/DL_rnn/rnn_custom_fast_dropout_025",
    "Keiiino/DL_rnn/rnn_handmade_dropout_none",
    # "Keiiino/DL_rnn/rnn_handmade_dropout_025"
]

# Login to wandb if you haven't already
wandb.login()

# Загрузка логов
api = wandb.Api()

# Fetch runs using correct paths
runs = []
for path in run_paths:
    try:
        run = api.run(path)
        runs.append(run)
    except wandb.errors.CommError as e:
        print(f"Warning: Could not find run at path: {path}")
        print(f"Error message: {e}")

# If any runs were not found, you can choose to skip them or handle it differently
if len(runs) != len(run_paths):
    print(f"Warning: {len(run_paths) - len(runs)} runs were not found. Plotting results for the found runs.")

# Continue with plotting if any runs were found
if runs:
    histories = [run.history(keys=["epoch", "train/loss", "val/acc"]) for run in runs]

    # Построение графиков
    fig, axes = plt.subplots(2, 1, figsize=(10, 10))
    colors = ['black', 'red', 'blue', 'green', 'orange', 'purple', 'cyan']

    for i, hist in enumerate(histories):
        axes[0].plot(hist["epoch"], hist["train/loss"], label=labels[i], color=colors[i])
        axes[1].plot(hist["epoch"], hist["val/acc"], label=labels[i], color=colors[i])

    axes[0].legend()
    axes[0].grid(True)
    axes[0].set_xlabel('Epoch')
    axes[0].set_title('CrossEntropy Loss')

    axes[1].legend()
    axes[1].grid(True)
    axes[1].set_xlabel('Epoch')
    axes[1].set_title('Accuracy')

    fig.tight_layout()
    plt.savefig("results.png")
    plt.show()
else:
    print("Error: No runs were found. Unable to plot results.")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Error message: Could not find run <Run Keiiino/DL_rnn/rnn_baseline (not found)>
Error message: Could not find run <Run Keiiino/DL_rnn/rnn_custom_dropout_none (not found)>
Error message: Could not find run <Run Keiiino/DL_rnn/rnn_custom_dropout_0_25 (not found)>
Error message: Could not find run <Run Keiiino/DL_rnn/rnn_custom_fast_dropout_none (not found)>
Error message: Could not find run <Run Keiiino/DL_rnn/rnn_custom_fast_dropout_025 (not found)>
Error message: Could not find run <Run Keiiino/DL_rnn/rnn_handmade_dropout_none (not found)>
Error: No runs were found. Unable to plot results.


Сделайте итоговые выводы о качестве работы моделей с разными реализациями DropOut:

**Ответ:**

## `Бонус. Zoneout (0.5 балла)`

Это еще одна модификация идеи дропаута применительно к рекуррентным нейросетям. В Zoneout на каждом временном шаге с вероятностью $p$ компонента скрытого состояния обновляется, а с вероятностью $1-p$ берется с предыдущего шага.
В Виде формул ($m^t_h$ - бинарная маска):

(сначала обычный рекуррентный переход, например LSTM)
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o)
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad
g = tanh(h_{t-1} W^g + x_t U^g+b_g)
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t)
$$
Затем Zoneout:
$$
h_t = h_t * m_h^t + h_{t-1}*(1-m_h^t)
$$
В этом методе маска уже должна быть разная во все моменты времени (иначе метод упрощается до дропаута Гала и Гарамани). На входы $x_t$ вновь можно накладывать маску до начала работы рекуррентного слоя.  

Если у вас осталось время, вы можете реализовать этот метод. Выберите основу из трех рассмотренных случаев самостоятельно.

**Полный балл ставится только при наличии качественного и количественного сравнения с предыдущими моделями.**

# `Часть 2. Language Modeling с помощью LSTM (5 баллов)`

Во второй части мы попробуем обучить модель для генерации отзывов по их началу.

Концептуально модель будет выглядеть следующим образом:
    
![image info](https://www.researchgate.net/publication/350391597/figure/fig1/AS:1005416683167744@1616721425265/Structure-of-the-long-short-term-memory-language-model-LSTMLM.png)

В процессе обучения будем тренировать сеть предсказывать вероятность следующего символа при условии всех предыдущих. Эту вероятность можно моделировать с помощью скрытого состояния $h^{(t)}$ пропуская его через линейный слой с выходной размерностью равной размерности словаря:
$$
p(x^{t}|x^{t-1}, ..., x^{1}) = SoftMax(Linear(h^{(t)}))
$$

Обратите внимание, что для вычисления $p(x^{t}|x^{t-1}, ..., x^{1})$ для всех моментов времени достаточно сделать один проход по RNN, а затем применить линейное преобразование ко всем скрытым состояниям.

В качестве функции потерь необходимо использовать `CrossEntropy`.

Рассмотрим другой важный момент. Для того, чтобы решить данную задачу, модель должна уметь определять момент начала генерации предложения и оповещать о завершении генерации — конце предложения. Для этого добавим в словарь вспомогательные токены `<sos>`, `<eos>`. Добавив `<sos>` в начало каждого предложения и `<eos>` в конец.

Модель сможет начинать генерацию как только ей будет передан токен `<sos>` и заканчивать генерацию, как только на очередном месте самым вероятным токеном оказывается `<eos>`.

Для решения этой задачи мы воспользуемся уже реализованной LSTM с дропаутом `FastRNNLayer` и классом `RNNClassifier`, то есть архитектура сети принципиально не поменяется.

## `Реализация модели и цикла обучения (2 балла)`

**Не используйте циклы в `RNNLM`, `LMCrossEntropyLoss`, `LMAccuracy`**

In [ ]:
class RNNLM(RNNClassifier):
    def __init__(
        self, embedding_dim, hidden_dim, tokenizer, dropout=0.5, layers_dropout=0.5, num_layers=1
    ):
        super().__init__(
            embedding_dim=embedding_dim, hidden_dim=hidden_dim,
            output_size=tokenizer.get_vocab_size(), tokenizer=tokenizer,
            rec_layer=FastRNNLayer, dropout=dropout, layers_dropout=layers_dropout, num_layers=num_layers
        )

    def forward(self, tokens, tokens_lens):
        """
        :param torch.Tensor(dtype=torch.long) tokens:
            Batch of texts represented with tokens. Shape: [T, B]
        :param torch.Tensor(dtype=torch.long) tokens_lens:
            Number of non-padding tokens for each object in batch. Shape: [B]
        :return torch.Tensor:
            Distribution of next token for each time step. Shape: [T, B, V], V — size of vocabulary
        """
        # Make embeddings for all tokens
        # [T, B, D] — получаем эмбеддинги
        embedded = self.embedding(tokens)

        # Forward pass embeddings through network
        # [T, B, H] — пропускаем через LSTM
        outputs, _ = self.rnn(embedded)

        # Take all hidden states from the last layer of LSTM for each step and perform linear transformation
        # [T, B, V] — преобразуем скрытые состояния в распределения по словарю
        logits = self.out(outputs)

        return logit

Реализуем функцию потерь для данной задачи.

Моменты на которые нужно обратить внимание:
1. Распределение вероятности следующего токена для последнего токена в последовательности не участвует в подсчёте функции потерь.
2. Необходимо учитывать, что в одном батче могут быть тексты разной длины.

Для решения второй проблемы можно воспользоваться функцией `torch.nn.utils.rnn.pack_padded_sequence`.

Принимая на вход батч тензоров и длину каждого тензора без учёта паддинга эта функция позволяет получить все элементы в тензорах, которые не относятся к паддингу в виде плоского массива:

In [70]:
padded_tensors = torch.tensor([
    [[1, 11, 111], [2, 22, 222], [3, 33, 333]],
    [[4, 44, 444], [5, 55, 555], [6, 66, 666]],
    [[7, 77, 777], [0, 0, 0], [8, 88, 888]],
    [[9, 99, 999], [0, 0, 0], [0, 0, 0]]
])
tensors_lens = torch.tensor([4, 2, 3])

Обратите внимание, что `torch.nn.utils.rnn.pack_padded_sequence` автоматически переупорядочивает тензоры в батче по убыванию их длины.

In [71]:
torch.nn.utils.rnn.pack_padded_sequence(padded_tensors, tensors_lens, batch_first=False, enforce_sorted=False)

PackedSequence(data=tensor([[  1,  11, 111],
        [  3,  33, 333],
        [  2,  22, 222],
        [  4,  44, 444],
        [  6,  66, 666],
        [  5,  55, 555],
        [  7,  77, 777],
        [  8,  88, 888],
        [  9,  99, 999]]), batch_sizes=tensor([3, 3, 2, 1]), sorted_indices=tensor([0, 2, 1]), unsorted_indices=tensor([0, 2, 1]))

In [3]:
class LMCrossEntropyLoss(torch.nn.CrossEntropyLoss):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, outputs, tokens, tokens_lens):
        """
        :param torch.Tensor outputs: Output from RNNLM.forward. Shape: [T, B, V]
        :param torch.Tensor tokens: Batch of tokens. Shape: [T, B]
        :param torch.Tensor tokens_lens: Length of each sequence in batch
        :return torch.Tensor: CrossEntropyLoss between corresponding logits and tokens
        """
        # Use torch.nn.utils.rnn.pack_padded_sequence().data to remove padding and flatten logits and tokens
        # Do not forget specify enforce_sorted=False and correct value of batch_first
        targets = tokens[1:]
        logits = outputs[:-1]

        # Sequence lengths for inputs and targets are reduced by 1
        adjusted_lengths = tokens_lens - 1

        # Pack logits and targets, flattening them and removing paddings
        packed_logits = torch.nn.utils.rnn.pack_padded_sequence(
            logits, adjusted_lengths.cpu(), enforce_sorted=False
        ).data  # shape: [sum(adjusted_lengths), V]

        packed_targets = torch.nn.utils.rnn.pack_padded_sequence(
            targets, adjusted_lengths.cpu(), enforce_sorted=False
        ).data  # shape: [sum(adjusted_lengths)]

        # Use super().forward(..., ...) to compute CrossEntropyLoss
        return super().forward(packed_logits, packed_targets)

Проверим как работает класс `LMCrossEntropyLoss`. Важно помнить, что при языковом моделировании мы предсказываем следующий токен, то есть `tokens` и `outputs` смещены друг относительно друга.

In [4]:
loss = LMCrossEntropyLoss(reduction='none')
max_len_testing    = 3
batch_size_testing = 2
vocab_size_testing = 5

logits = torch.zeros((max_len_testing, batch_size_testing, vocab_size_testing))
tokens = torch.zeros((max_len_testing, batch_size_testing)).long()
lens   = torch.tensor([2, 3])

logits[:, 0, 0] = torch.tensor([1, -1, 0])
logits[:, 0, 1] = torch.tensor([2, 3,  0])
logits[:, 0, 2] = torch.tensor([3, 2,  0])
logits[:, 0, 3] = torch.tensor([4, 2,  0])
logits[:, 0, 4] = torch.tensor([5, 2,  0])


logits[:, 1, 0] = torch.tensor([-1, 10, 2])
logits[:, 1, 1] = torch.tensor([2, 30,  1])
logits[:, 1, 2] = torch.tensor([4, 20,  1])
logits[:, 1, 3] = torch.tensor([5, -10, 4])
logits[:, 1, 4] = torch.tensor([1, 7,  13])

tokens[:, 0]    = torch.tensor([1, 4, 0])
tokens[:, 1]    = torch.tensor([3, 1, 4])


loss(outputs=logits, tokens=tokens, tokens_lens=lens)

tensor([ 3.3636,  0.4519, 23.0000])

Для оценки качества нам также необходимо вычислять долю правильно предсказанных токенов. Реализуйте класс для вычисления точности.

In [5]:
class LMAccuracy(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, outputs, tokens, tokens_lens):
        """
        :param torch.Tensor outputs: Output from RNNLM.forward. Shape: [T, B, V]
        :param torch.Tensor tokens: Batch of tokens. Shape: [T, B]
        :param torch.Tensor tokens_lens: Length of each sequence in batch
        :return torch.Tensor: Accuracy for given logits and tokens
        """
        # Use torch.nn.utils.rnn.pack_padded_sequence().data to remove padding and flatten logits and tokens
        # Do not forget specify enforce_sorted=False and correct value of batch_first
        # YOUR CODE HERE
        targets = tokens[1:]
        logits = outputs[:-1]

        adjusted_lengths = tokens_lens - 1

        packed_logits = torch.nn.utils.rnn.pack_padded_sequence(
            logits, adjusted_lengths.cpu(), enforce_sorted=False
        ).data  # shape: [N, V]

        packed_targets = torch.nn.utils.rnn.pack_padded_sequence(
            targets, adjusted_lengths.cpu(), enforce_sorted=False
        ).data  # shape: [N]

        # Получаем предсказания: индекс токена с максимальной вероятностью
        predictions = packed_logits.argmax(dim=-1)

        # Считаем точность
        correct = (predictions == packed_targets).sum()
        total = packed_targets.numel()

        return correct.float() / total

Проверим как работает класс `LMAccuracy`.

In [6]:
metric = LMAccuracy()
max_len_testing    = 3
batch_size_testing = 2
vocab_size_testing = 5

logits = torch.zeros((max_len_testing, batch_size_testing, vocab_size_testing))
tokens = torch.zeros((max_len_testing, batch_size_testing)).long()
lens   = torch.tensor([2, 3])

logits[:, 0, 0] = torch.tensor([1, -1, 0])
logits[:, 0, 1] = torch.tensor([2, 3,  0])
logits[:, 0, 2] = torch.tensor([3, 2,  0])
logits[:, 0, 3] = torch.tensor([4, 2,  0])
logits[:, 0, 4] = torch.tensor([5, 2,  0])


logits[:, 1, 0] = torch.tensor([-1, 10, 2])
logits[:, 1, 1] = torch.tensor([2, 30,  1])
logits[:, 1, 2] = torch.tensor([4, 20,  1])
logits[:, 1, 3] = torch.tensor([5, -10, 4])
logits[:, 1, 4] = torch.tensor([1, 7,  13])

tokens[:, 0]    = torch.tensor([1, 4, 0])
tokens[:, 1]    = torch.tensor([3, 1, 4])


metric(outputs=logits, tokens=tokens, tokens_lens=lens)

tensor(0.3333)

In [8]:
metric = LMAccuracy()
max_len_testing    = 10
batch_size_testing = 3
vocab_size_testing = 200

set_global_seed(42)
logits = torch.randn((max_len_testing, batch_size_testing, vocab_size_testing))
logits[:, :, 1] = 1000

tokens = torch.randint(low=0, high=vocab_size_testing, size=(max_len_testing, batch_size_testing))
tokens[:5, :] = 1
lens   = torch.tensor([10, 4, 9])

assert metric(outputs=logits, tokens=tokens, tokens_lens=lens) == (4 + 3 + 4) / (lens - 1).sum()

Модифицируйте функции `train_epoch`, `evaluate`, `train` для обучения LM.

**При вычислении точности, обратите внимание на то, что мы не предсказываем первый токен в каждой последовательности и токены, относящиеся к паддингу.**

In [9]:
def train_epoch_lm(dataloader, model, loss_fn, optimizer, device):
    model.train()
    for tokens, tokens_lens in dataloader:
        tokens = tokens.to(device)
        tokens_lens = tokens_lens.to(device)

        optimizer.zero_grad()

        outputs = model(tokens, tokens_lens)
        loss = loss_fn(outputs, tokens, tokens_lens)

        loss.backward()
        optimizer.step()


def evaluate_lm(dataloader, model, loss_fn, device):
    model.eval()

    total_tokens = 0
    total_loss = 0.0
    total_accuracy = 0.0

    accuracy_fn = LMAccuracy()

    with torch.no_grad():
        for tokens, tokens_lens in dataloader:
            tokens = tokens.to(device)
            tokens_lens = tokens_lens.to(device)

            outputs = model(tokens, tokens_lens)

            loss = loss_fn(outputs, tokens, tokens_lens)
            acc = accuracy_fn(outputs, tokens, tokens_lens)

            total_loss += loss.item() * (tokens_lens.sum().item() - tokens.size(1))  # -1 per sequence
            total_accuracy += acc.item() * (tokens_lens.sum().item() - tokens.size(1))
            total_tokens += (tokens_lens - 1).sum().item()

    return total_loss / total_tokens, total_accuracy / total_tokens


def train_lm(
    train_loader, test_loader, model, loss_fn, optimizer, device, num_epochs
):
    test_losses = []
    train_losses = []
    test_accuracies = []
    train_accuracies = []
    for epoch in range(num_epochs):
        train_epoch_lm(train_loader, model, loss_fn, optimizer, device)

        train_loss, train_acc = evaluate_lm(train_loader, model, loss_fn, device)
        train_accuracies.append(train_acc)
        train_losses.append(train_loss)

        test_loss, test_acc = evaluate_lm(test_loader, model, loss_fn, device)
        test_accuracies.append(test_acc)
        test_losses.append(test_loss)

        print(
            'Epoch: {0:d}/{1:d}. Loss (Train/Test): {2:.3f}/{3:.3f}. Accuracy (Train/Test): {4:.3f}/{5:.3f}'.format(
                epoch + 1, num_epochs, train_losses[-1], test_losses[-1], train_accuracies[-1], test_accuracies[-1]
            )
        )
    return train_losses, train_accuracies, test_losses, test_accuracies

Теперь у нас всё готово для обучения модели.

Создадим токенизатор с `<sos>`, `<eos>` токенами.

Обратите внимание, что в отличие от классификации текстов нам необходимо значительно увеличить размер словаря, чтобы доля `<unk>` токенов была не велика.

Так же, так как задача генерации значительно сложнее задачи классификации текстов будем обучать модель только на префиксах рецензий длины $20$. Это позволяет значительно ускорить обучение.

In [10]:
# min_freq=8 is approximately equivalent to max_size=30000.
#   You can lower min_freq in order to make model vocabulary more diverse
trainer = trainers.WordLevelTrainer(
    min_frequency  = 8,
    special_tokens = ['<pad>', '<unk>', '<sos>', '<eos>'],
)

tokenizer = tokenizers.Tokenizer(
    model = tokenizers.models.WordLevel(unk_token="<unk>")
)

NameError: name 'trainers' is not defined

In [ ]:
tokenizer.train_from_iterator(tqdm(get_data_iterator(), total=50_000), trainer=trainer)

In [ ]:
assert tokenizer.get_vocab_size() == 30000

for i in range(10):
    token = tokenizer.id_to_token(i)
    print(f"ID = {i}, token = {token}")


In [ ]:
lm_test_dataset = LargeMovieReviewDataset(test_data_path, tokenizer, max_len=20, pad_sos=True, pad_eos=True)
lm_train_dataset = LargeMovieReviewDataset(train_data_path, tokenizer, max_len=20, pad_sos=True, pad_eos=True)

Создадим даталоадеры для тестовой и обучающей выборок:

In [ ]:
lm_test_dataloader = DataLoader(
    lm_test_dataset, batch_size=196, shuffle=False, num_workers=4,
    collate_fn=partial(collate_fn, padding_value=tokenizer.token_to_id('<pad>'))
)
lm_train_dataloader = DataLoader(
    lm_train_dataset, batch_size=196, shuffle=True, num_workers=4,
    collate_fn=partial(collate_fn, padding_value=tokenizer.token_to_id('<pad>'))
)

Убедитесь, что все предложения имеют в начале `<sos>` токен, а в конце — `<eos>` токен.

In [ ]:
batch = next(iter(lm_train_dataloader))
batch['tokens'], batch['tokens_lens']

Создадим модель, функцию потерь и оптимизатор:

In [ ]:
lm_model = RNNLM(
    embedding_dim=512, hidden_dim=512, tokenizer=tokenizer, dropout=0.6, layers_dropout=0.6, num_layers=2
).to(device=device)

In [ ]:
lm_loss_fn = LMCrossEntropyLoss(reduction='mean')
lm_optimizer = torch.optim.Adam(lm_model.parameters(), lr=0.005, weight_decay=1.2e-6)

Обучим модель:

In [ ]:
%%time
lm_model = torch.compile(lm_model)

In [ ]:
lm_train_losses, lm_train_accuracies, lm_test_losses, lm_test_accuracies = train_lm(
    lm_train_dataloader, lm_test_dataloader, lm_model, lm_loss_fn, lm_optimizer, device, 10
)

## `Реализация декодера (1 балл)`

Теперь, реализуем последнюю деталь — декодирование с использованием обученной модели.
Есть несколько вариантов. Рассмотрим два самых простых:
1. **Жадное декодирование.** На каждом шаге мы выбираем токен с максимальной вероятностью и используем его для обновления скрытого состояния RNN.
2. **Top-k sampling.** На очередном шаге рассматриваются $k$ токенов с самыми большими вероятностями. Остальные токены игнорируются. Из выбранных токенов семплируется следующий токен пропорционально их вероятностям.

Прочитать подробнее про разные варианты декодирования можно по ссылкам:
1. [От huggingface](https://huggingface.co/blog/how-to-generate)
2. [На towardsdatascience](https://towardsdatascience.com/decoding-strategies-that-you-need-to-know-for-response-generation-ba95ee0faadc)

Существенным в процессе декодирования является критерий останова. Как только очередной самый вероятный символ оказался `<eos>`, то данная последовательность считается сгенерированной. Однако, может так оказаться, что `<eos>` никогда не будет выбран, тогда необходимо прекратить генерацию, как только длина последовательности перейдёт порог `max_generated_len`.

In [11]:
@torch.no_grad()
def decode(
    model, start_tokens, start_tokens_lens,
    max_generated_len=20, top_k=None, eos_idx=tokenizer.token_to_id('<eos>')
):
    """
    :param RNNLM model: Model
    :param torch.Tensor start_tokens: Batch of seed tokens. Shape: [T, B]
    :param torch.Tensor start_tokens_lens: Length of each sequence in batch. Shape: [B]
    :param int max_generated_len: Maximum length of generated samples
    :param Optional[int] top_k: Number of tokens with the largest probability to sample from
    :return Tuple[torch.Tensor, torch.Tensor]. Newly predicted tokens and length of generated part.
    """
    device = start_tokens.device
    embedding = model.embedding(start_tokens)

    # Pass through RNN step by step to collect final hidden and cell state
    all_h, all_c = [], []
    h = embedding.new_zeros([model.rnn.num_layers, start_tokens.shape[1], model.hidden_dim])
    c = embedding.new_zeros([model.rnn.num_layers, start_tokens.shape[1], model.hidden_dim])

    for t in range(start_tokens.shape[0]):
        output, (h, c) = model.rnn(embedding[t].unsqueeze(0), (h, c))
        all_h.append(h)
        all_c.append(c)

    all_h = torch.stack(all_h, dim=1)
    all_c = torch.stack(all_c, dim=1)

    # Select final h, c according to lengths
    last_indices = start_tokens_lens - 1
    batch_indices = torch.arange(start_tokens.shape[1], device=device)
    h = all_h[:, last_indices, batch_indices]  # shape: [num_layers, B, hidden]
    c = all_c[:, last_indices, batch_indices]

    predicted_tokens = []
    decoded_lens = torch.zeros_like(start_tokens_lens)
    is_finished_decoding = torch.zeros_like(start_tokens_lens, dtype=torch.bool)

    # Initialize input token as the last token from start_tokens
    input_token = start_tokens[-1]

    for _ in range(max_generated_len):
        embedding = model.embedding(input_token).unsqueeze(0)  # Shape: [1, B, D]
        output, (h, c) = model.rnn(embedding, (h, c))           # Shape: [1, B, H]

        logits = model.output_layer(output.squeeze(0))         # Shape: [B, V]

        if top_k is not None:
            # Top-k sampling
            topk_vals, topk_idx = torch.topk(logits, top_k, dim=-1)
            probs = torch.nn.functional.softmax(topk_vals, dim=-1)
            next_token = topk_idx.gather(1, torch.multinomial(probs, 1)).squeeze(1)
        else:
            # Greedy decoding
            next_token = torch.argmax(logits, dim=-1)          # Shape: [B]

        predicted_tokens.append(next_token)

        # Update generation status
        decoded_lens += (~is_finished_decoding)
        is_finished_decoding |= (next_token == eos_idx)

        if torch.all(is_finished_decoding):
            break

        input_token = next_token

    return torch.stack(predicted_tokens), decoded_lens


NameError: name 'tokenizer' is not defined

Для тестирования создадим удобный класс `FakeLM`, который будет реализовывать весь необходимый функционал, а именно:

- `word_embeddings` - по токенам возвращает их эмбеддинги;

- `output` - по вектору контекста (h) возвращает вероятности следующих токенов;

- `rnn` - обновляет контекст. Будем использовать класс `FakeRNN`.

Для простоты будем считать, что эмбеддинг токена и есть сам токен, а процесс генерации является марковским, то есть все переходы можно описать матрицей.

Такая постановка поможет нам удобно задать все переходы и нарисовать их на бумаге (если возникнет необходимость в отладке). Кроме того, мы можем не думать об обновлении контекста, так как контекст и есть текущее слово (его эмбеддинг).

In [12]:
class FakeRNN:

    def __init__(self):

        self.num_layers = 1

    def __call__(self, embedding, h_c):
        """
        :param torch.Tensor embedding: Batch of embedding. Shape: [T, B, E]
        :param Optional[Tuple[torch.Tensor, torch.Tensor]] h_c: initial hidden state and initial cell state
        """

        return None, (embedding, torch.zeros_like(embedding))


class FakeLM:

    def __init__(self, rnn, transition_matrix):
        self.rnn = rnn
        self.transition_matrix = transition_matrix
        self.hidden_dim = 1

    def word_embeddings(self, tokens):
        """
        :param torch.Tensor tokens: Batch of seed tokens. Shape: [T, B] or [B]
        :return torch.Tensor
            Batch of tokens embeddings. Shape: [T, BS, E] or [B, E]
        """

        return tokens[..., None].float()

    def output(self, h):
        """
        :param torch.Tensor h: Batch of seed tokens. Shape: [B, E]
        :return torch.Tensor.
            Batch of new_tokens logits. Shape: [B, V]
        """

        idx  = h.long().ravel().cpu().numpy()
        logits = list(self.transition_matrix[i] for i in idx)
        logits = torch.stack(logits, dim=0).float()
        return logits



#### `Тест 1`

Зададим переходную матрицу `transitions` так, чтобы после токена $i$ вероятней всего следовал токен $i + 1$ mod $V$, где $V$ размер словаря. Тогда мы ожидаем получить циклические последовательности.

In [13]:
transitions = {
    0: torch.tensor([4, 5, 0, 0, 0]),
    1: torch.tensor([0, 4, 5, 0, 0]),
    2: torch.tensor([0, 0, 4, 5, 0]),
    3: torch.tensor([0, 0, 0, 4, 5]),
    4: torch.tensor([5, 0, 0, 0, 4]),
}

fake_lm = FakeLM(FakeRNN(), transitions)

In [14]:
decoded_tokens, decoded_lens = decode(
    model             = fake_lm,
    start_tokens      = torch.zeros((4, 3)),
    start_tokens_lens = torch.tensor([1, 2, 3]),
    eos_idx           = 10,
    max_generated_len = 8
)


NameError: name 'decode' is not defined

In [ ]:
assert decoded_tokens.shape == (8, 3), f"Shape = {decoded_tokens.shape}"
assert (decoded_tokens[:, 0] == torch.tensor([1, 2, 3, 4, 0, 1, 2, 3])).all()
assert (decoded_tokens[:, 1] == torch.tensor([1, 2, 3, 4, 0, 1, 2, 3])).all()
assert (decoded_tokens[:, 2] == torch.tensor([1, 2, 3, 4, 0, 1, 2, 3])).all()

#### `Тест 2`

Усложним и зададим разные начальные токены.

In [ ]:
start_tokens = torch.tensor(
    [
        [0, 0, 0],
        [0, 1, 1],
        [0, 0, 2],
        [0, 0, 0]
    ]
)

decoded_tokens, decoded_lens = decode(
    model             = fake_lm,
    start_tokens      = start_tokens,
    start_tokens_lens = torch.tensor([1, 2, 3]),
    eos_idx           = 10,
    max_generated_len = 8
)

In [ ]:
assert decoded_tokens.shape == (8, 3), f"Shape = {decoded_tokens.shape}"
assert (decoded_tokens[:, 0] == torch.tensor([1, 2, 3, 4, 0, 1, 2, 3])).all()
assert (decoded_tokens[:, 1] == torch.tensor([2, 3, 4, 0, 1, 2, 3, 4])).all()
assert (decoded_tokens[:, 2] == torch.tensor([3, 4, 0, 1, 2, 3, 4, 0])).all()

In [ ]:
decoded_tokens, decoded_lens = decode(
    model             = fake_lm,
    start_tokens      = start_tokens,
    start_tokens_lens = torch.tensor([1, 2, 3]),
    eos_idx           = 4,
    max_generated_len = 8
)
assert (decoded_lens == torch.tensor([4, 3, 2])).all()

assert (decoded_tokens[:, 0] == torch.tensor([1, 2, 3, 4])).all()
assert (decoded_tokens[:, 1] == torch.tensor([2, 3, 4, 0])).all()
assert (decoded_tokens[:, 2] == torch.tensor([3, 4, 0, 1])).all()

#### `Тест 3`

Перейдем к тестированию `top_k`. Благодаря этому параметру мы можем увеличить разнообразие текстов. Поскольку проверять недетерминированную генерацию достаточно трудно, мы проверим, что в среднем генерируются адекватные последовательности.

При заданной нами переходной матрице мы ожидаем:

- При `top_k = 1` получим детерминированную генерацию;

- При `top_k = 2` получим последовательности, где в почти в половине случаев будем стоять на месте;

- При `top_k > 2` получим похожее на `top_k = 2`.

In [ ]:
set_global_seed(42)

decoded_tokens, decoded_lens = decode(
    model             = fake_lm,
    start_tokens      = torch.zeros((4, 3)),
    start_tokens_lens = torch.tensor([1, 2, 3]),
    eos_idx           = 10,
    max_generated_len = 8,
    top_k             = 1
)

In [ ]:
assert decoded_tokens.shape == (8, 3), f"Shape = {decoded_tokens.shape}"
assert (decoded_tokens[:, 0] == torch.tensor([1, 2, 3, 4, 0, 1, 2, 3])).all()
assert (decoded_tokens[:, 1] == torch.tensor([1, 2, 3, 4, 0, 1, 2, 3])).all()
assert (decoded_tokens[:, 2] == torch.tensor([1, 2, 3, 4, 0, 1, 2, 3])).all()

In [ ]:
set_global_seed(42)

decoded_tokens_k2, decoded_lens = decode(
    model             = fake_lm,
    start_tokens      = torch.zeros((4, 3)),
    start_tokens_lens = torch.tensor([1, 2, 3]),
    eos_idx           = 10,
    max_generated_len = 8,
    top_k             = 2
)

decoded_tokens_k2

In [ ]:
set_global_seed(42)

decoded_tokens_k3, decoded_lens = decode(
    model             = fake_lm,
    start_tokens      = torch.zeros((4, 3)),
    start_tokens_lens = torch.tensor([1, 2, 3]),
    eos_idx           = 10,
    max_generated_len = 8,
    top_k             = 3
)

In [ ]:
assert (decoded_tokens_k3 == decoded_tokens_k2).all()

Попробуем сгенерировать продолжения для нескольких префиксов:

In [ ]:
start_tokens = torch.tensor([
    lm_model.tokenizer.encode(['<sos>', '<pad>', '<pad>', '<pad>'], is_pretokenized=True).ids,
    lm_model.tokenizer.encode(['<sos>', 'my', 'favorite', 'movie'], is_pretokenized=True).ids,
    lm_model.tokenizer.encode(['<sos>', 'the', 'best', 'movie'],    is_pretokenized=True).ids,
    lm_model.tokenizer.encode(['<sos>', 'the', 'worst', 'movie'],   is_pretokenized=True).ids,
]).T

start_tokens_lens = torch.tensor([1, 4, 4, 4])
start_tokens

In [ ]:
set_global_seed(42)

lm_model = lm_model.cpu()
lm_model.eval()
decoded_tokens, decoded_lens = decode(lm_model, start_tokens, start_tokens_lens, max_generated_len=10, top_k=5)

In [ ]:
for text_idx in range(start_tokens.shape[1]):
    decoded_text_tokens = decoded_tokens[:decoded_lens[text_idx], text_idx]
    tokens = start_tokens[:start_tokens_lens[text_idx], text_idx].tolist() + decoded_text_tokens.tolist()
    words = lm_model.tokenizer.decode(tokens, skip_special_tokens=False).split(' ')

    text = ' '.join(words).replace('<', '&lt;').replace('>', '&gt;')
    display(Markdown(f'<div class="alert alert-block alert-info"> <b>{text}</b></div>'))

Попробуйте выполнить семплирование для разных $k$. Сравните результаты top-k семплирования с жадным декодированием. Опишите ваши наблюдения.

In [ ]:
# YOUR CODE HERE

**Ответ:**

## `Beam Search (2 балла)`

Рассмотрим более продвинутый алгоритм для декодирования. Реализуйте алгоритм Beam Search.

Прочитать подробнее про Beam Search можно по ссылкам:
1. [От huggingface](https://huggingface.co/blog/how-to-generate#beam-search)
2. [На wiki](https://en.wikipedia.org/w/index.php?title=Beam_search&oldid=1248868876)

Несколько замечаний по имплементации:

1. При больших размерах `beam_size` число гипотез ($B \times \text{beam\_size}$) на очередном шаге может быть слишком большим. Поэтому может потребоваться разбить все гипотезы на отдельные батчи и делать forward-pass в несколько итераций. Используйте [`torch.split`](https://pytorch.org/docs/stable/generated/torch.split.html)
2. Для выбора лучших гипотез используйте [`torch.topk`](https://pytorch.org/docs/stable/generated/torch.topk.html). Обратите внимание на индексы, которые возвращает эта функция (может пригодиться метод [`torch.remainder`](https://pytorch.org/docs/stable/generated/torch.remainder.html))
3. Можно отслеживать, какие элементы в батче (или какие гипотезы) закончили генерацию. Делая forward-pass только для незавершённых гипотез, можно ускорить декодинг, однако, это усложнит реализацию

Будем реализовывать в 2 части:

- Получение всей необходимой информации из `start_tokens` в функции `beam_search_encode_start`.

- Генерация и декодирования в функции `decode_beam_search`.

In [ ]:
@torch.no_grad()
def beam_search_encode_start(model, start_tokens, start_tokens_lens, beam_size):
    """
    :param RNNLM model: Model
    :param torch.Tensor start_tokens: Batch of seed tokens. Shape: [T, B]
    :param torch.Tensor start_tokens_lens: Length of each sequence in batch. Shape: [B]
    :param int beam_size: Size of beam
    :return Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor].
        Initialization h and c for each hypotheses, New tokens,
        Initialize log probabilities, Initialize hypothesis
        Shape [L, B * BS, H], [L, B * BS, H], [B * BS], [B * BS], [1, B * BS]
    """
    # L — number of RNN layers in the model, H — hidden size, BS — beam size
    #
    # 1. Make forward pass of start_tokens through the model.
    #      Obtain the last cell and hidden state for each element in the batch
    #          (i.e. tensors of shape [L, B, H])
    #      Use those states as the initialization for each hypotheses in the beam
    #          (i.e. tensors of shape [L, B * BS, H])
    #      Initialize probabilities for each hypotheses in the beam with 1.0
    #          (i.e. tensor of shape [B * BS])

    # Get embeddings for the start tokens
    # YOUR CODE HERE
    ...

    # Make forward pass through the RNN and
    #   obtain the last cell and hidden state for each element in the batch
    # YOUR CODE HERE
    start_h = ... # [L, B, H]
    start_c = ... # [L, B, H]


    # Use those states as the initialization for each hypotheses in the beam
    # YOUR CODE HERE
    h = ... # [L, B * BS, H]
    c = ... # [L, B * BS, H]

    # Select initial tokens for each hypotheses in the beam
    #   Compute log probabilities and select top-beam_size tokens for each element
    #   Use them to initialize beam search state
    # YOUR CODE HERE
    ...

    new_tokens = ... # [B * BS]
    log_probas = ... # [B * BS]
    hypothesis = ... # [1, B * BS]

    return h, c, new_tokens, log_probas, hypothesis

Проверим функцию `beam_search_encode_start`. Ниже опишем матрицу перехода, в которой содержатся логиты вероятности перехода. `Beam size` отвечает за число гипотез, то есть сколько потенциально лучших токенов мы можем взять. Например, при `beam size = 4` и начальном токене $0$, мы будем рассматривать (создадим гипотезы) токены $[1, 0, 4, 3]$ (отсортированы в порядке убывания вероятности).

In [ ]:
transitions = {
    0: torch.tensor([4, 5, 1, 2, 3]),
    1: torch.tensor([3, 4, 5, 1, 2]),
    2: torch.tensor([2, 3, 4, 5, 1]),
    3: torch.tensor([1, 2, 3, 4, 5]),
    4: torch.tensor([5, 1, 2, 3, 4]),
}

fake_lm = FakeLM(FakeRNN(), transitions)

In [ ]:
start_tokens = torch.tensor(
    [
        [0, 0, 0],
        [0, 1, 1],
        [0, 0, 2],
        [0, 0, 0]
    ]
)

beam_size = 4
h, c, new_tokens, log_probas, hypothesis = beam_search_encode_start(
    model             = fake_lm,
    start_tokens      = start_tokens,
    start_tokens_lens = torch.tensor([1, 2, 3]),
    beam_size         = beam_size
)

In [ ]:
assert h.shape == (1, 3 * beam_size, 1)
assert c.shape == (1, 3 * beam_size, 1)
assert new_tokens.shape[0] == (3 * beam_size)
assert log_probas.shape[0] == (3 * beam_size)
assert hypothesis.shape     == (1, 3 * beam_size)

In [ ]:
assert (h[0, :, 0] == torch.tensor([0] * beam_size + [1] * beam_size + [2] * beam_size)).all()
assert (new_tokens == torch.tensor([1, 0, 4, 3, 2, 1, 0, 4, 3, 2, 1, 0])).all()
assert torch.isclose(log_probas[:beam_size], torch.tensor([5, 4, 3, 2, 1]).float().log_softmax(0)[:-1]).all()
assert (hypothesis[0] == new_tokens).all()

In [ ]:
@torch.no_grad()
def decode_beam_search(
    model, start_tokens, start_tokens_lens,
    max_generated_len=20, beam_size=5, eos_idx=tokenizer.token_to_id('<eos>')
):
    """
    :param RNNLM model: Model
    :param torch.Tensor start_tokens: Batch of seed tokens. Shape: [T, B]
    :param torch.Tensor start_tokens_lens: Length of each sequence in batch. Shape: [B]
    :param int max_generated_len: Maximum length of generated samples
    :param int beam_size: Size of beam
    :return Tuple[torch.Tensor, torch.Tensor, torch.Tensor].
        Newly predicted tokens, lengths of generated parts and log probabilities for each hypotheses
        Shape [T*, B, beam_size], [T*, beam_size], [T*, beam_size]
    """

    # L — number of RNN layers in the model, H — hidden size, BS — beam size
    #
    # 1. Make forward pass of start_tokens through the model.
    #      Initialize vector that show whether hypothesis is finished
    #          (i.e. tensor of shape [B * BS])
    # 2. While all sequences do not end with <eos> and their length less than max_generated_len
    #      1. Get probabilities for the next token for each hypothesis
    #          (i.e. tensor of shape [B * BS, V])
    #      2. Use those probabilities to compute probability for each extension of each hypothesis
    #          (i.e. tensor of shape [B * BS, V])
    #      3. For each element in the batch select new BS best hypotheses
    #          Note, that some of the hypotheses on the previous step have been finished
    #            so their probability should not change. So you have to select BS best hypotheses
    #            among all extension of unfinished hypotheses and finished hypotheses
    #          As a result you will have a new token for best extensions of unfinished hypotheses
    #          For simplisity you can use <EOS> token if you select finished hypothesis in the beam
    #            i.e. tensor of shape [B * BS] of indices for selected hypotheses and
    #                 tensor of shape [B * BS] of extension tokens for each hypothesis
    #      4. Update probabilities for each hypotheses and is_finished state for each hypothesis
    #          Concat new tokens to the existing prefixes
    #      5. Update hidden and cell state to correspond to the selected hypothesis

    #  Make forward pass of start_tokens through the model.
    h, c, new_tokens, log_probas, hypothesis = beam_search_encode_start(
        model             = model,
        start_tokens      = start_tokens,
        start_tokens_lens = start_tokens_lens,
        beam_size         = beam_size
    )

    # Initialize vector that show whether hypothesis is finished
    is_finished = new_tokens == eos_idx # [B * BS]
    decoded_lens = torch.ones_like(is_finished, dtype=torch.long) # [B * BS]

    while not torch.all(is_finished) and hypothesis.shape[0] < max_generated_len:
        # Get probabilities for the next token for each hypothesis
        # YOUR CODE HERE
        ...

        next_token_log_probas = ... # [B * BS, V]
        # Use those probabilities to compute probability for each extension of each hypothesis
        # YOUR CODE HERE
        ...
        extension_log_probas = ... # [B * BS, V]

        # For each element in the batch select new BS best hypotheses
        #   You can use loop over different beams
        # YOUR CODE HERE
        ...

        # Update probabilities for each hypotheses and is_finished state and decoded_lens for each hypothesis
        # YOUR CODE HERE
        ...

        # Concat new tokens to the existing prefixes
        # YOUR CODE HERE
        ...

        # Update hidden and cell state to correspond to the selected hypothesis
        # YOUR CODE HERE
        ...


    return (
        hypothesis.view(-1, start_tokens.shape[1], beam_size),
        decoded_lens.view(start_tokens.shape[1], beam_size),
        log_probas.view(start_tokens.shape[1], beam_size)
    )

Протестируем итоговую функцию `decode_beam_search`. Рассмотрим постановку, когда действовать жадно не выгодно для генерации.

In [ ]:
transitions = {
    0: torch.tensor([-1e10, 1, -1e10, -1e10, 0.9]),
    1: torch.tensor([-1e10, 1, 1.1, 0.9, -1e10]),
    2: torch.tensor([-1e10, 0.9, 1, 1.1, -1e10]),
    3: torch.tensor([-1e10, 1.1, 0.9, 1, -1e10]),
    4: torch.tensor([-1e10, -1e10, -1e10, -1e10, 1]),
}

fake_lm = FakeLM(FakeRNN(), transitions)

In [ ]:
for k, v in transitions.items():
    p = v.softmax(0).cpu().numpy()
    p = np.around(p, 3)
    print(f"log p(next_token | {k}) = {p}")

Действуя жадно и начиная с токена $0$, мы перейдем в токен $1$, а дальше с вероятностью будем блуждать по цепочке $1 \to 2 \to 3 \to 1$. Но при числе гипотез больше 2 мы рассмотрим возможность перехода в токен $4$, где зациклимся. В результате во втором случае у последовательности будет высокая итоговая вероятность, а у первого способа - низкая.

Сначала проверим как работает `beam_size = 1`, так как этот случай проще отладить в случае ошибки.

**Важно:** При `beam_size = 1` мы получаем жадное декодирование.

In [ ]:
set_global_seed(42)

decoded_tokens, decoded_lens, log_probas = decode_beam_search(
    model             = fake_lm,
    start_tokens      = torch.zeros((1, 3)),
    start_tokens_lens = torch.tensor([1, 1, 1]),
    eos_idx           = 10,
    max_generated_len = 8,
    beam_size         = 1
)

p = transitions[1].log_softmax(0)[2] * 7 + transitions[0].log_softmax(0)[1]

In [ ]:
assert decoded_tokens.shape == (8, 3, 1)
assert decoded_lens.shape   == (3, 1)
assert log_probas.shape     == (3, 1)

In [ ]:
decoded_tokens[:, :, 0]

In [ ]:
assert torch.isclose(log_probas, p).all()

In [ ]:
set_global_seed(42)

decoded_tokens, decoded_lens, log_probas = decode_beam_search(
    model             = fake_lm,
    start_tokens      = torch.zeros((1, 3)),
    start_tokens_lens = torch.tensor([1, 1, 1]),
    eos_idx           = 10,
    max_generated_len = 8,
    beam_size         = 2
)


In [ ]:
assert decoded_tokens.shape == (8, 3, 2)
assert decoded_lens.shape   == (3, 2)
assert log_probas.shape     == (3, 2)

Посмотрим на наиболее вероятную гипотезу для каждого батча:

In [ ]:
decoded_tokens[:, :, 0]

Посмотрим на вторую гипотезу для каждого батча:

In [ ]:
decoded_tokens[:, :, 1]

Подробнее, как это работает:

1. На первом ходу мы берем 2 наиболее вероятных перехода: перейти в $1$, перейти в $4$.

2. На следующих ходах мы рассматриваем для каждого "луча" 2 наиболее вероятных перехода (всего 4 случаев), из которых выбираем две наиболее вероятные гипотезы.

- 1. Для случая, который перешел в $1$, мы рассматриваем 2 альтернативы: перейти в $2$ с вероятностью $0.367$, остаться в $1$ с вероятностью $0.332$)
- 2. Для случая, который перешел в $4$, мы рассматриваем 2 альтернативы: остаться в $4$ с вероятностью 1, или перейти в любое другое состояние с вероятностью $\approx$ 0)
- 3. Из всех вариантов вероятней всего перейти из $1$ в $2$ и перейти из $4$ в $4$, то есть обновляем "лучи"

3. Повторить пункт 2.

**Важно:** Обратите внимание, что Beam Search детерминированный алгоритм

In [ ]:
start_tokens = torch.tensor([
    lm_model.tokenizer.encode(['<sos>', '<pad>', '<pad>', '<pad>'], is_pretokenized=True).ids,
    lm_model.tokenizer.encode(['<sos>', 'my', 'favorite', 'movie'], is_pretokenized=True).ids,
    lm_model.tokenizer.encode(['<sos>', 'the', 'best', 'movie'],    is_pretokenized=True).ids,
    lm_model.tokenizer.encode(['<sos>', 'the', 'worst', 'movie'],   is_pretokenized=True).ids,
]).T

start_tokens_lens = torch.tensor([1, 4, 4, 4])

In [ ]:
lm_model.to(device).eval()
start_tokens = start_tokens.to(device)
start_tokens_lens = start_tokens_lens.to(device)

In [ ]:
beam_size = 100
decoded_tokens, decoded_lens, log_probas = decode_beam_search(
    lm_model, start_tokens, start_tokens_lens, max_generated_len=10, beam_size=beam_size
)

In [ ]:
for start_tokens_elem, start_tokens_lens_elem, decoded_tokens_elem, decoded_lens_elem, log_probas_elem in zip(
    start_tokens.T, start_tokens_lens,
    decoded_tokens.permute(1, 2, 0), decoded_lens.permute(0, 1), log_probas.permute(0, 1)
):
    start_tokens_elem = start_tokens_elem[:start_tokens_lens_elem].tolist()
    start_words = lm_model.tokenizer.decode(start_tokens_elem, skip_special_tokens=False).split(' ')

    start_text = ' '.join(start_words).replace('<', '&lt;').replace('>', '&gt;')
    display(Markdown(f'<div class="alert alert-block alert-info"> <b>{start_text}</b></div>'))

    for idx, (hyp, hyp_len, hyp_log_prob) in enumerate(zip(decoded_tokens_elem, decoded_lens_elem, log_probas_elem)):
        if idx >= 3:
            break

        hyp = hyp[:hyp_len].tolist()
        hyp_words = lm_model.tokenizer.decode(hyp, skip_special_tokens=False).split(' ')
        hyp_text = ' '.join(hyp_words).replace('<', '&lt;').replace('>', '&gt;')
        display(Markdown(
            f'<div class="alert alert-block alert-success"> <b>{hyp_log_prob:.3f}: {hyp_text}</b></div>'
        ))

Попробуйте выполнить декодинг для разных `beam_size`. Убедитесь, что при `beam_search=1` семплирование совпадает с top-1 (greedy decoding) подходом.

Сравните результаты Beam Search с top-k семплированием и жадным декодированием. Опишите ваши наблюдения.

In [ ]:
# YOUR CODE HERE

## `Бонус. Существенное улучшение качества (до 6 баллов)`

Та модель, которая использовалась в предыдущей части во многом заимствует улучшения LSTM из статьи [Regularizing and Optimizing LSTM Language Models](https://arxiv.org/pdf/1708.02182.pdf). Вы можете попробовать применить другие варианты регуляризации из данной статьи для существенного улучшения качества LM.

Например:
1. Dropout для эмбеддингов **(+0.25)**
2. Dropout входов и выходов RNN **(+0.25)**
3. Регуляризация активаций (AR/TAR) **(+1.0)**
4. NT-ASGD **(+1.5)**
5. Tied веса эмбеддингов и софтмакса **(+1.0)**
6. Attention **(+2.0)**

**Полные баллы ставятся только при наличии качественного и количественного сравнения с бейзлайном.**

**Для эксперимента с Attention необходимо изобразить Attention Maps для нескольких примеров.**